**Description:**
This Jupyter notebook is developed to show case the use of JSON header files for faster data access to NWM files. Here's a brief overview of the imported modules:

- `joblib`: Used for parallel processing and efficient caching.
- `numpy`: A fundamental library for numerical operations.
- `xarray`: A library for working with labeled multi-dimensional arrays, often used in scientific data analysis.
- `fsspec`: Provides a common interface for working with various filesystem-like protocols.
- `ujson`: A fast JSON encoder and decoder for handling JSON data.
- `matplotlib.pyplot`: Used for creating data visualizations.
- `psutil`: Provides information on system resource utilization.
- `concurrent.futures`: A module for asynchronously executing functions using threads or processes.
- `multiprocessing`: A library for parallel and concurrent computing.
- `urlgennwm`: An external library/module for generating URLs specific to the NWM (National Water Model).



In [ ]:
import joblib
import numpy as np
import xarray as xr
import fsspec
import ujson
import matplotlib.pyplot as plt
import psutil
import nwmurl
import os

# 1: Processing Zarr files

This code defines a set of functions for working with remote datasets in JSON format. It includes functions to load remote JSON content, open datasets from JSON, select streamflow data, and select time values. The `process_file` function combines these operations to process a single file for a given feature ID, making it useful for retrieving and working with specific data from remote sources.

In [ ]:
# Define a function to load remote JSON content
def load_remote_json(file_url):
    of = fsspec.open(file_url)
    with of as f:
        return ujson.load(f)

# Define a function to load a remote dataset from JSON content
def load_remote_ds(json_obj):
    backend_args = {
        "consolidated": False,
        "storage_options": {
            "fo": json_obj,
        },
    }
    return xr.open_dataset("reference://", engine="zarr", backend_kwargs=backend_args)

# Define a function to select streamflow data from a dataset
def select_flow(ds, feature_id):
    cords = ds.streamflow.sel(feature_id=feature_id)
    return cords.values

# Define a function to select time
def select_time(ds):
    dstime = ds.time
    return dstime.values

# Define a function to process a single file for a given feature ID
def process_file(file_url, feature_id):
    try:
        json_obj = load_remote_json(file_url)
        ds = load_remote_ds(json_obj)
    except Exception as e:
        print(f"Error processing Zarr file: {e}")
        return None, None
    streamflow_value = select_flow(ds, feature_id)
    time_value = select_time(ds)
    return streamflow_value, time_value


**Feature ID Selection:**
This code defines a list of feature IDs, which are typically used as identifiers for specific geographic or data features. In this case, the list includes feature IDs `8153461`, `8153027`, and `18210860`, representing specific features of interest. These feature IDs are used in subsequent data retrieval. Feature IDs could for different locations could be found at https://water.noaa.gov/map#forecast-chart


In [ ]:
# Define a list of 1 feature ID
feature_ids = [8153461]

In [ ]:
# Define a list of 12,000 feature IDs
feature_ids = [
    22811611, 23832907, 21412883, 25293410, 15183793, 18524217, 626220, 4726685, 15048261, 2297884, 9074086, 14353046, 167200737, 2788603, 9660756, 4539621, 10055266, 10452202, 7077392, 8835386, 24903800, 3125212, 18122652, 20105349, 7703046, 15714785, 6232986, 11160893, 24719331, 3766342, 4438382, 1523997, 167578935, 10975909, 1737140, 3172512, 4608830, 6742822, 26815201, 2245123, 1199836, 2236109, 1010002391, 6318401, 800227, 5205320, 939010255, 1970745, 2702541, 16659839, 1643861, 9897866, 8200407, 23940171, 1020000221, 9619605, 948090631, 25055589, 8479948, 11284660, 9182098, 15888878, 10367158, 7718580, 3712048, 24270972, 17623995, 54524, 22561410, 6399779, 22599807, 17155414, 17170112, 10390202, 9316927, 946050038, 17573699, 6170754, 24279018, 166195783, 898449, 1440477, 17665749, 7913958, 17159214, 6134087, 1174452, 25388456, 22393819, 163864378, 8272933, 6251384, 23856583, 20156487, 5846422, 5197182, 16817635, 15842570, 20324441, 946060337, 10736883, 18122654, 11909762, 20268261, 13069184, 19833722, 25371895, 9409382, 25151247, 9455643, 165536688, 23241390, 6264182, 946020430, 946040234, 8394180, 11279938, 9630389, 7917743, 800351, 18082798, 943030266, 23980371, 18257965, 22653169, 23914361, 17173900, 19784451, 23184739, 2805097, 23955760, 946060297, 7088581, 6724863, 7843189, 947120092, 6130319, 20296931, 20226573, 2172729, 22549123, 24044347, 18077192, 21415111, 946060015, 16841444, 11690260, 9305350, 22599827, 10499569, 22525185, 10717998, 20296925, 24274343, 24035960, 22226684, 16944504, 21906573, 17150414, 23909577, 17889966, 23228564, 8317553, 15108627, 2681538, 16683705, 15852025, 8835906, 23889434, 20156405, 4767502, 16875696, 20342653, 23871829, 18077224, 946060194, 10719408, 10249847, 22518294, 11677813, 19887653, 24057183, 11363705, 19876012, 947120082, 19809535, 11690116, 20156425, 15448486, 20331522, 14301857, 18253067, 24281956, 21420492, 17587726, 18522615, 2678762, 10519476, 23876101, 5868233, 20832408, 23838450, 17599293, 9486502, 2544457, 943020181, 23881060, 1524153, 5290259, 8457872, 6251382, 24537862, 20204804, 9891216, 1985180, 8955774, 946060404, 23232963, 22677316, 16683861, 2287399, 17562700, 19779797, 20119322, 17693919, 11411875, 2678346, 11392210, 22634746, 6318405, 24017699, 2665669, 9629899, 9633737, 19875206, 6141214, 20830552, 17165784, 23977614, 1670527, 20332084, 10680592, 17605441, 8281731, 5205314, 9903163, 20664952, 27609963, 946020037, 20829490, 11338089, 24026228, 8315605, 16696427, 10018420, 20849799, 5205436, 1609020, 23844621, 23886240, 11124787, 16910627, 1189927, 11397894, 20372843, 24950581, 15886142, 8386123, 10499583, 946020018, 946030196, 20694370, 948090871, 23844949, 19802987, 20331702, 20315416, 23949563, 948091000, 22823956, 17706191, 23880632, 6178614, 23864738, 948070199, 7733853, 2785533, 21270906, 5845728, 10452332, 817141, 17715352, 1657155, 11154815, 23838154, 9436873, 2676686, 25068048, 19808461, 20828334, 6317747, 1563610, 20828690, 23875749, 9454915, 25227992, 945010387, 17889656, 4769258, 20350775, 9902873, 91972, 8403917, 23997022, 20205274, 23230457, 21376156, 5876319, 10694707, 945050107, 10402153, 8386087, 25227986, 20161695, 1615834, 1615424, 9409500, 17136566, 10960362, 24061111, 946030192, 10961534, 6322233, 9905081, 3340772, 945040599, 8956940, 19875066, 85494, 16918808, 24884094, 11211036, 24750045, 943030292, 9660752, 948100283, 23864566, 5867753, 207523, 18293484, 23864346, 11364667, 9907127, 11079361, 15182421, 21276680, 4769480, 945040428, 11411935, 2785525, 14598365, 11712119, 5860787, 946060056, 24285484, 11566106, 14299889, 22595517, 933120030, 946030127, 24748285, 20259695, 6724895, 946030437, 10682662, 17657279, 20351619, 16884000, 24416444, 1737130, 24405993, 2665659, 22771162, 948080332, 20156409, 11341567, 15108501, 20322614, 15857821, 22383931, 91896, 21246849, 7733919, 946020032, 19860136, 5881881, 17715840, 22630780, 478975, 944040176, 11331405, 6261976, 22832490, 8077930, 23174495, 23266884, 11690040, 9302509, 8457486, 20315378, 9356790, 17170500, 10479911, 15448512, 20287722, 23860815, 14599103, 1576436, 24043295, 17174066, 9888492, 8377193, 947120081, 8377087, 15842578, 8955720, 10320326, 9356820, 1150000432, 946050252, 14962483, 11079471, 15448570, 25335151, 23227722, 1180000368, 24986466, 20827754, 20366017, 24061225, 17695739, 17620670, 19779171, 9409350, 6325325, 17784675, 22632344, 2678314, 9620927, 20206688, 8080210, 8075804, 11332193, 946060026, 6724819, 22628532, 19817777, 18522729, 21901875, 20357348, 1670659, 11323174, 9350061, 2785543, 9985193, 20851687, 1176504, 22770726, 24664243, 8955610, 6318651, 19810805, 23997332, 2663277, 10000042, 22634828, 90276, 4767400, 20331222, 17763025, 17762827, 7734457, 948091108, 22678450, 20156411, 2179049, 5329465, 7734453, 10734452, 1670505, 23920848, 5298501, 17684066, 14598347, 15838510, 24074049, 10484491, 24042897, 11323142, 2666085, 18516172, 8075784, 931010009, 8377301, 21902907, 24057349, 22675792, 948090314, 8384655, 2665655, 16933198, 11682705, 1174064, 948100344, 816959, 21261463, 1574704, 22677258, 1173978, 15182449, 24286938, 22383963, 6261880, 8415649, 1588293, 24059699, 1986372, 22826484, 1588433, 21303225, 946060407, 22824802, 79568, 10682220, 947020269, 16865399, 24004189, 946030197, 19875302, 1203566, 946020694, 2676304, 5205416, 2078119, 16056680, 944040013, 948091005, 20161183, 8193037, 23480043, 19810021, 22818166, 7737521, 7733789, 1692028, 11125963, 8952900, 25205493, 20463512, 17763909, 9627041, 19770071, 478983, 22681136, 11331601, 8075926, 25227998, 22636296, 20667056, 4535989, 19856992, 948030279, 19805597, 11677449, 12059272, 11847523, 10720352, 9468645, 17162490, 23990739, 17174242, 22677310, 20867464, 24059437, 9903411, 6178248, 17688301, 20161393, 10715174, 24534276, 209929, 8460400, 6130363, 24399968, 9496576, 945080061, 943030256, 18301766, 19804859, 22678110, 22674764, 10679680, 8188863, 25088399, 22393265, 9455269, 17150266, 23175431, 8077778, 19804237, 14596673, 22825666, 946020680, 2685846, 23174579, 24985951, 79494, 18292058, 948090294, 8953266, 17785633, 19808487, 20225955, 24993052, 946030178, 20849783, 23920486, 2677338, 24072853, 20381085, 1202128, 2179057, 5846372, 11284796, 8191209, 8384581, 5203188, 8402125, 10695761, 21263015, 24061485, 1187877, 19859432, 14746536, 23920590, 2544113, 18515690, 24073929, 20206628, 22677178, 10681212, 1692576, 11387425, 945010434, 14301859, 210073, 21422342, 11322692, 18524207, 7732095, 8317425, 20372769, 11323116, 23185065, 10721056, 24018069, 17136570, 10960084, 22556276, 6170768, 23064777, 1895144, 17161802, 11125529, 22594055, 19885619, 11682865, 5849520, 20370163, 19834008, 24075249, 17716264, 9903385, 17161990, 946060042, 1660732, 20664770, 940250172, 1224166, 6322315, 22771778, 8409163, 946020391, 9442313, 11396822, 1170352, 15183681, 20849803, 19858272, 2804929, 9486540, 14596609, 9350101, 2820036, 19856430, 16770844, 17163478, 15843620, 9486530, 23881192, 2245041, 23871145, 11154155, 17764087, 68160, 2676164, 20288160, 9454953, 11335711, 24235258, 9409608, 11363593, 19854688, 16770620, 165536606, 15842080, 20223823, 10715564, 23949491, 9985211, 22631384, 10367150, 946020381, 21411921, 3060088, 9411826, 24279634, 5863919, 21263783, 17715880, 6242077, 23860751, 8190405, 24114607, 22592741, 14593473, 23893333, 20866922, 22636314, 20866576, 478985, 9455667, 10499605, 20310541, 5876789, 20155935, 25136425, 23233193, 13068942, 20341997, 16867953, 943020351, 8404151, 1587365, 8317451, 1566732, 24027276, 24026222, 24543503, 5205406, 20866850, 24987721, 8192913, 8315911, 1692590, 14599361, 20296829, 19834506, 23700521, 1408915, 11362381, 20206614, 20394863, 22678136, 17168860, 19854370, 24416186, 948010239, 5860733, 16865421, 24270130, 20859643, 9629875, 2680262, 20505538, 24042977, 23238358, 24416154, 5863827, 478959, 5845954, 11083109, 9627119, 10477277, 946060039, 11690174, 24074107, 24004113, 22822762, 10696061, 8190705, 10681036, 943060125, 23910139, 24017767, 24534406, 22763420, 20206664, 8188763, 6322273, 24993002, 6314353, 5329303, 6178200, 22822350, 800863, 9436881, 24116087, 2685766, 1174178, 23649466, 20829494, 11391943, 22591993, 11710737, 22457335, 166174925, 21400467, 11301932, 12086415, 24013911, 946030006, 23277126, 20849771, 11083131, 17687971, 23844901, 11566130, 6318391, 1987464, 24001089, 17174194, 8457534, 23838802, 947040064, 945060024, 6321767, 21258613, 22763046, 21902919, 8402071, 24521430, 11083037, 8457566, 9437503, 12639785, 11314261, 24287110, 5802200, 13069192, 21258591, 22625352, 26816607, 16852144, 11079905, 23989681, 14747434, 2804921, 948030126, 23185173, 948090653, 22821304, 2678254, 9411858, 948080370, 6178292, 20853937, 17174062, 16976854, 17171526, 9630001, 948090553, 24286866, 15111445, 17596211, 19886523, 8075626, 23015074, 945040019, 21307129, 8265330, 11202844, 10678018, 9890510, 17160180, 4769234, 948091293, 19836148, 15108385, 21901859, 11083515, 2236037, 1171000044, 17692863, 7071682, 11396892, 10408300, 9486342, 940150012, 19817761, 11361131, 22628458, 10233327, 946020440, 2826308, 22680842, 17763133, 17171416, 19853802, 16768178, 20370149, 17150226, 2664353, 21271364, 17593975, 6177670, 2179363, 20852149, 18292228, 945030392, 17171608, 2685488, 2676552, 19779027, 1559026, 6141190, 24287250, 20287720, 17594763, 20215658, 12086807, 8412899, 6314355, 6178258, 22626124, 10695757, 10232583, 946020045, 18292206, 2556397, 8315833, 20371899, 22599815, 25209248, 948080349, 5867683, 5863879, 23949869, 11359833, 11359797, 945050457, 24989657, 22628540, 19810133, 4604270, 21309043, 8193581, 11332473, 8376189, 11321844, 2665639, 19833670, 20287718, 22384449, 20351625, 23952728, 19888339, 6318029, 22630778, 6177936, 16918736, 22672546, 166196264, 948100672, 10466421, 17150284, 15838112, 19834966, 24287050, 10715412, 17727290, 5863953, 9987669, 22634546, 20864943, 24057451, 17171264, 8190321, 19854240, 10674520, 945010529, 12087689, 478529, 9455445, 22226680, 17508012, 23864522, 17763871, 7866486, 9627247, 24044045, 943050172, 21270370, 19886453, 20867010, 90188, 19808575, 23015704, 21303283, 11601577, 9630051, 8317415, 24043869, 22598713, 22595615, 11323186, 9486400, 9483638, 241840, 25079148, 19770051, 24416630, 10719440, 12085753, 24400026, 18301782, 11157777, 1523809, 15154634, 22624614, 23015170, 11209882, 17431668, 9626907, 17154470, 946060052, 948091318, 948091078, 948090493, 4763216, 19834490, 24481627, 23096902, 25909319, 22629208, 2179179, 6241841, 24001239, 5845762, 20852991, 8922047, 20259151, 17684380, 10407324, 22595367, 14597359, 24028970, 23886196, 13071938, 13071932, 20341963, 24073957, 19875264, 22824094, 19808539, 11157025, 17705011, 17695615, 17170634, 2683658, 2682714, 20505496, 20635168, 9350011, 10853991, 24279666, 5860941, 92074, 6181312, 9455337, 19808573, 11083161, 1185247, 17171594, 17171408, 7733097, 25159732, 1564528, 20830348, 25088439, 20827524, 11324108, 5876557, 24028368, 24027120, 24286836, 21269990, 24543807, 11566034, 6321837, 12085185, 21258647, 12606187, 9455285, 9455079, 23015234, 22681742, 8457786, 2682718, 1566394, 24044145, 16768632, 1693330, 2804607, 11715503, 24558757, 2664773, 18516196, 17716154, 17715196, 19808611, 3880064, 23015166, 20848697, 26812379, 8457836, 945020420, 945080021, 18073080, 24035962, 948091191, 14593337, 20752495, 23957436, 22383895, 18517152, 2788393, 23233283, 2785099, 22771790, 5863413, 90168, 19876560, 19810027, 20853949, 17705107, 24060429, 6168954, 23860687, 945030353, 24575424, 20380517, 17688375, 1566400, 24043039, 20827480, 10734426, 9356802, 18073088, 9486486, 20363903, 19836212, 20161671, 10721006, 24416582, 20156537, 22383879, 4164820, 6325425, 24019053, 12088275, 8415685, 19810123, 9455163, 20529403, 11151963, 21309329, 11210026, 7734421, 17169554, 10480075, 10480043, 948100727, 24235618, 948091103, 948091006, 10854255, 18516206, 22769238, 8410579, 945050224, 19810763, 9454971, 2179073, 10695819, 9449971, 11283052, 24652013, 17704789, 10233197, 2561953, 22677318, 655480, 7733327, 19856642, 17693891, 2683592, 22603481, 11394668, 948100469, 20371391, 17682686, 946060315, 22393295, 19836078, 10721120, 23233299, 12087895, 23227896, 20603388, 8410651, 6181326, 20268395, 24001053, 10236761, 23864486, 938080296, 17162310, 21744637, 23186361, 11191551, 22069698, 14598397, 15450136, 14598103, 3060098, 10002748, 1671833, 11565346, 11364435, 9985189, 24074765, 943020212, 26432086, 19810057, 166741121, 20331466, 11154291, 24061131, 18290200, 17889954, 3340088, 2682954, 11332261, 9888470, 17620668, 10738095, 17682760, 24559467, 14596655, 2661999, 24026212, 943030448, 22771672, 20215500, 19887693, 946040294, 11363251, 945050429, 9455249, 6178290, 939010232, 20261007, 21373876, 8462044, 9903807, 24057505, 17764481, 8189139, 20311471, 11332767, 933160080, 19784335, 11392556, 11321876, 7913900, 946060329, 946060023, 8954434, 10718124, 20350539, 6321931, 13069064, 20866740, 17716224, 945050181, 15878569, 19875674, 20854615, 19804749, 10693379, 2827986, 938090255, 8460420, 7734439, 19854128, 16770856, 22599561, 20828922, 14600495, 14599457, 7913972, 4766994, 22591671, 6131459, 23886520, 21261725, 92060, 24664245, 16864709, 933120029, 9455383, 149054, 7093737, 23475838, 23864296, 11082929, 20781392, 25227988, 11137600, 10677980, 18082748, 948100350, 20372511, 11393532, 2676916, 11849875, 14600497, 23251133, 7913898, 11125145, 4767432, 24558657, 2665647, 5876505, 24287202, 21534286, 6721531, 24543585, 1670349, 10320546, 20864947, 24076127, 22762450, 9455573, 23480051, 3359364, 11878252, 5846376, 20849779, 16850890, 9903769, 18291786, 9902597, 22681120, 7733099, 17168986, 22673852, 1636535, 16053968, 19855034, 19854490, 948100543, 20831056, 20830440, 448726, 16767174, 17684386, 20633342, 18073102, 7913968, 11713519, 10466003, 2664703, 20228341, 24026816, 24026782, 20223377, 11964799, 15108407, 90134, 24075431, 22762448, 19810703, 9455193, 22824036, 2179091, 20463616, 11678721, 20853065, 10694651, 20852719, 945040042, 946020434, 18291894, 24451963, 19074499, 11601473, 16056616, 23920426, 12056596, 17168404, 2684282, 9629919, 10480109, 15852951, 11395596, 11393966, 23838864, 948091267, 22657083, 948090878, 6724891, 11312117, 11310921, 6263394, 22187643, 21400471, 23954059, 17596149, 20805835, 91928, 11362151, 17783237, 24074473, 22763268, 20206842, 24073197, 1986338, 1984542, 19809891, 2179145, 19806093, 3879720, 11154141, 10693747, 19866002, 11082289, 1185307, 17763777, 8192993, 19853904, 19853810, 19853760, 14412663, 11854001, 7913954, 20693398, 10402357, 11712209, 15967999, 19833998, 6263190, 10715510, 14181592, 22635682, 20865724, 17785069, 8410687, 20599416, 22625398, 10232311, 14491852, 938090113, 948050049, 2298068, 8457922, 14746902, 14681150, 19858106, 11338403, 17889936, 22673422, 166196253, 9629803, 16770748, 11396376, 17162942, 2675298, 11323192, 11125393, 22593399, 946060327, 948091187, 15837121, 12296630, 11311011, 19828484, 11564496, 23227856, 24406367, 11364641, 17590180, 24074781, 22630490, 22825254, 6178144, 20464038, 11157007, 5846350, 11677401, 23997546, 21374872, 21374618, 20848627, 18291912, 8264644, 948050054, 2297862, 19073725, 7866506, 17171444, 24575366, 19856392, 8190665, 3340490, 12056652, 2682762, 16768574, 11394510, 11323182, 24298178, 11387500, 9486312, 4767436, 14597135, 20625570, 8956974, 19244591, 9411902, 19829488, 20351687, 15107469, 10715318, 10320800, 22771758, 10319808, 16937918, 24407631, 20341911, 19810661, 9455209, 2179329, 3880554, 23015214, 23997282, 20849871, 23864462, 24058135, 948050072, 22681168, 11277328, 20319818, 11339625, 11338109, 10681002, 8190201, 7071702, 11395660, 16767142, 1692826, 23186461, 18073462, 9356772, 20365153, 17150530, 15837009, 3711714, 6724963, 2067968, 6130475, 10782430, 15107765, 6129003, 22444030, 13071848, 23228464, 13069780, 17590276, 21062348, 16867799, 24534444, 20601998, 24074959, 20666868, 20206632, 1987346, 16863641, 18960606, 19875138, 8403927, 14300549, 11547306, 11154055, 17705821, 19802921, 6170530, 20849709, 20848733, 19995361, 20781154, 18290166, 23269606, 9630505, 11398844, 20635500, 948100614, 948100468, 940170352, 11852393, 2677174, 23254099, 1692578, 22598443, 23253379, 10406978, 14599089, 14598839, 14596723, 948090598, 8956948, 9349569, 24750051, 15967965, 11311041, 20288204, 10719606, 23499121, 943030306, 12087973, 20867440, 6318047, 11362427, 24534530, 16865415, 24074151, 16928236, 22824796, 19875656, 3161853, 22821836, 24001179, 21903639, 21903353, 16723722, 10693741, 10234553, 23732917, 20848797, 11083503, 9903647, 1187229, 948050047, 1185151, 9442413, 9901081, 22612943, 7866712, 14352934, 24116079, 12056760, 22673498, 16971066, 9499898, 8187663, 17684084, 22595295, 15975980, 9356776, 24233092, 24821889, 15842560, 946060386, 946060017, 948090556, 948090543, 945010224, 20296997, 17150406, 22393213, 1131000003, 7056895, 21276268, 25077314, 23893303, 9409560, 23499143, 10718510, 10718296, 12683647, 10715772, 1671765, 23952646, 12745197, 15889810, 24406617, 20277769, 12085491, 16934932, 22635138, 20666108, 20665578, 1987624, 22825334, 933120093, 18302040, 21906347, 10961550, 15154214, 1587435, 16918890, 16918760, 23865354, 24061201, 21308469, 24058355, 24057713, 17174236, 17762461, 20381073, 9305354, 19855872, 11335931, 17624457, 20368902, 948030103, 22596159, 24561019, 24036180, 11387303, 10402351, 25212196, 22655471, 8957460, 19834466, 6262962, 3897129, 5860759, 946040101, 19885129, 92144, 22636366, 6317479, 89172, 22630826, 8409063, 16928272, 19876616, 17844640, 21905749, 19808587, 22821864, 20854351, 19804819, 23277672, 18292872, 10753089, 22680558, 22940954, 13503446, 17172568, 20382201, 12713744, 9436775, 20380537, 19855922, 10482365, 19853936, 944040179, 11396806, 16770518, 2483128, 20832366, 20635648, 21225261, 22601313, 12639497, 1693174, 1693170, 17159012, 11848903, 24560747, 5882755, 22592151, 11123249, 24885103, 940160350, 6141206, 17150312, 21409580, 15836801, 24028148, 24027536, 24027316, 1678819, 7969413, 5018814, 23499167, 23499153, 23957396, 10717978, 943030425, 17596161, 17594785, 13071996, 23096034, 9987677, 91480, 7299940, 11689806, 24534568, 24075211, 9984201, 8409157, 22825256, 18303036, 3359236, 948060135, 23410029, 20851809, 945040046, 19802647, 24453831, 946020025, 9509774, 18291156, 18290902, 23336316, 10753095, 17763223, 17762241, 8193593, 22676546, 8192493, 24575436, 7732155, 2685510, 9498912, 2683112, 2682874, 11333489, 11398960, 11396812, 11396350, 16770104, 20505622, 11395026, 20635134, 2676306, 2676104, 1692058, 19779779, 948030239, 23251165, 22595721, 903049, 23185155, 18072974, 11124667, 946060380, 948091270, 5878625, 17150362, 24293750, 19835054, 17475485, 15181343, 19834128, 10853999, 23174225, 24288104, 23500101, 6722833, 22187389, 4032868, 24543411, 17595317, 19887733, 23949927, 13069478, 24013751, 20277037, 24208237, 8412833, 20864663, 20864651, 6314271, 7952272, 8409177, 1984918, 7947692, 83358, 20267019, 19807747, 14301853, 19806841, 8075704, 9449545, 945040593, 945040039, 11479092, 11282430, 21309199, 6170358, 14885661, 10232411, 24060403, 20781958, 20715674, 24057479, 17763611, 23989261, 9503988, 8192511, 20513244, 12058594, 17169082, 16971550, 20640350, 11334229, 11397868, 1174084, 10480107, 16770734, 17162634, 11394530, 17161598, 10409208, 20829292, 8377171, 20370225, 8376875, 10407548, 23186925, 21744405, 21744237, 24233486, 11388064, 9618383, 11124715, 22658079, 946060294, 10402349, 948091242, 5617434, 5878623, 21279288, 4763918, 15838412, 20229203, 22916074, 15836925, 15836847, 19834338, 21275990, 14918611, 19833776, 15048425, 2662037, 19832430, 10395213, 25729732, 23238538, 10000058, 11113773, 10720350, 909020389, 942050069, 3572855, 24019439, 17136726, 948010044, 10320532, 816965, 19887605, 17593191, 6318657, 11364265, 23881076, 24339374, 14639445, 936010022, 89926, 24074379, 24073983, 24073485, 22631538, 24073193, 24007630, 20664792, 16863597, 1986408, 84156, 9455229, 19809707, 1524333, 6241839, 24001147, 23475804, 11155021, 10761769, 20852917, 23997528, 11283100, 24061021, 24060369, 24059237, 16849216, 948050070, 24057779, 23860661, 22680870, 2692002, 21303995, 20779060, 23990053, 23989471, 23988949, 17762807, 17762407, 2819932, 22938996, 11207860, 7734445, 10683126, 8192559, 24575410, 19856600, 3341160, 20379637, 15857423, 9500052, 19854440, 10678594, 9498890, 2682888, 2485722, 7269341, 8317563, 2483174, 17162552, 948100563, 11852747, 23844559, 1692588, 4769240, 11322704, 7913904, 7913902, 22069030, 24558841, 23706614, 948090995, 11121459, 20228819, 23241026, 23240002, 19042967, 21532906, 21270210, 909020568, 20155261, 10716068, 10715352, 21527928, 10320574, 17594801, 19888323, 20215686, 20215316, 24407871, 4942247, 11363005, 22635156, 16933154, 6314349, 22632398, 24008528, 23483253, 17716084, 1986348, 9455687, 19875498, 4605158, 22624538, 4339752, 19806095, 8074642, 23277720, 20851915, 23276404, 16918814, 21374758, 21308431, 18293180, 9903417, 21437107, 948050113, 8460386, 8460292, 22681518, 8459890, 22681162, 21303789, 23990277, 17370874, 19074447, 22678212, 14485894, 17762411, 11601513, 9503992, 14746672, 19858342, 19858180, 9633681, 17694015, 17169302, 11467566, 20641730, 24114191, 23064775, 2683042, 2682732, 10678004, 11397292, 16770768, 2680472, 2680230, 11198452, 11852077, 15980763, 11393240, 11392596, 18077182, 1692564, 20828570, 943050183, 1558954, 18073442, 14599119, 11125113, 11713529, 24558363, 948091314, 899379, 17152184, 22591657, 20625538, 20296909, 5878883, 10466113, 14593217, 19835822, 305759, 20228429, 19834372, 24028100, 18521837, 11312759, 24026716, 23174567, 21270976, 12683661, 2787595, 166758533, 6719541, 10716998, 24019179, 17594783, 17594775, 22444068, 20477270, 13069528, 24406481, 2386056, 948070189, 91570, 22635070, 16867787, 24469852, 11689936, 20536370, 23156668, 24073745, 24073651, 23877035, 22631478, 11358931, 11227686, 478543, 19089749, 22825282, 19809637, 19809571, 3359518, 11157439, 22822778, 20331564, 22821108, 11548666, 21903007, 20526175, 11284740, 20393873, 11283916, 945040616, 8203249, 21375558, 17705159, 10233379, 1188259, 6168846, 14819407, 16850836, 24059185, 11082997, 24452219, 17176078, 266755, 20779042, 17763409, 23989325, 13109053, 1574680, 17171666, 17170654, 22675566, 7732439, 14678462, 17692957, 23066865, 943060230, 1176652, 17296752, 9497344, 5302335, 8316881, 16770708, 11396664, 9627097, 23977880, 1564564, 1170764, 11393998, 16767848, 11852041, 10409212, 10408342, 7917751, 932040119, 18072876, 14599321, 947110189, 15842740, 10402257, 948091296, 948090998, 948090789, 945010548, 19904002, 5878897, 15181341, 10003408, 11311497, 9999836, 21532912, 21270358, 1213923, 10715718, 2785391, 943030252, 943030166, 11172867, 10320776, 10320640, 23886104, 816417, 21328723, 6318015, 22636262, 16934526, 6316589, 23224536, 22632464, 945050090, 6312191, 16928630, 1986324, 16928296, 19810653, 1984624, 9455053, 19875158, 19808723, 22626166, 9451235, 5846362, 20853109, 9449785, 9449527, 5844742, 945040425, 22882083, 10232347, 11411879, 9445649, 18292944, 7151300, 21437955, 24452379, 16849778, 16849432, 9902369, 14752018, 16977876, 23989341, 11340663, 23988991, 22678102, 11340213, 9438919, 19858026, 17170418, 20512724, 13500198, 24575416, 13499404, 10484713, 24116091, 19856200, 3341104, 10681006, 11336237, 8190143, 946010265, 15857435, 13891082, 9500154, 2682990, 2551709, 1174352, 24046103, 1173982, 11593653, 20637460, 20571705, 10675542, 16770152, 17163216, 26927614, 11394646, 2678116, 22535378, 11852731, 1694114, 23844599, 10408868, 19780279, 8376901, 11850247, 19779615, 17812774, 11324190, 22595645, 11323126, 18072938, 18072862, 18072576, 22070040, 24035978, 20365117, 5881561, 4767152, 24558665, 22592527, 14596537, 15448480, 24557937, 948090877, 24885167, 9352959, 948090479, 945010123, 9483646, 11121491, 22393221, 20557713, 15183565, 1683065, 24357147, 25208958, 15836807, 1221930, 19834124, 24028080, 23241150, 19832432, 25206039, 9999820, 21271862, 21271024, 11112277, 948080334, 22187705, 11177525, 22384031, 10718196, 17599211, 6130309, 12683655, 2787071, 20350703, 24544941, 17335875, 20742256, 10715240, 11303558, 9991999, 13071966, 19887655, 1406319, 12087589, 946040078, 946040039, 92090, 92056, 24733510, 20800661, 21258763, 936010043, 8281847, 16604259, 16865431, 24073877, 12342308, 23876569, 22762262, 1200078, 1986358, 23480737, 7947998, 6964419, 19874934, 19809295, 15876719, 22823266, 1523951, 22822502, 8403963, 24001509, 22821320, 11155347, 20854307, 5846476, 22819862, 20853645, 943010002, 20852791, 10235931, 11677269, 19802831, 945040047, 20261503, 17705383, 20850241, 3154807, 24322890, 17703511, 10232401, 10232339, 24060263, 20258687, 24452759, 16849630, 9508446, 23860645, 17372538, 10818306, 14749856, 20516599, 23989795, 11210104, 23989355, 16975430, 7734423, 10683102, 16974268, 16056594, 23920818, 17170284, 11206490, 9436865, 8190623, 23066797, 9500086, 9499872, 19853778, 11202910, 20639746, 24505282, 10479909, 7071706, 1566646, 20440120, 2483216, 11854039, 24961156, 10477245, 945020418, 948100545, 940170351, 17880926, 22599495, 1693160, 1692606, 21746559, 19779543, 943050171, 1886294, 11126001, 24297996, 5881821, 14597201, 24558645, 946060069, 948091043, 20297463, 20297447, 948090482, 24294928, 20624566, 10466233, 10466041, 17150332, 21410070, 22654751, 1224386, 15836967, 19833594, 24027710, 25074886, 11312095, 3971176, 6722655, 12554640, 19828890, 10718498, 10718376, 942050113, 11110591, 16222359, 3572723, 15106269, 21527886, 20151361, 10320950, 10320916, 23886162, 10713256, 6322211, 20215734, 13137649, 19887667, 12088267, 13069826, 6318605, 5203366, 21258779, 17784997, 1203344, 6314249, 16865407, 22632444, 23876789, 1987530, 25842605, 22630828, 19876680, 1985254, 19810669, 1984582, 19809731, 19875250, 20071773, 3359774, 24986425, 20529663, 7881200, 1982932, 940130431, 15155150, 3161887, 22822418, 14826983, 19806067, 10696185, 9449943, 10695091, 20852571, 20851595, 11676219, 20392021, 22227002, 8202215, 21374184, 19014527, 10232423, 2236049, 11279930, 9903507, 24452045, 22681792, 18290126, 22812087, 22680284, 16977666, 14945930, 17763149, 17762431, 17762299, 1574366, 22676596, 22676506, 19858348, 19857796, 3997906, 10680972, 10680768, 22673500, 17495525, 19854262, 8384667, 10678274, 17690523, 2551573, 10677992, 19852146, 17164320, 20834260, 17688327, 17687921, 11396406, 16770012, 17163028, 948100733, 20635172, 20372713, 11394246, 20372609, 20830676, 8378245, 11392202, 11850329, 23514612, 10734688, 948030122, 20694872, 11126501, 24036402, 20693652, 11845825, 22593557, 20365189, 17809274, 23838430, 24558687, 22592373, 946060045, 17807898, 1226898, 948090708, 948090654, 948090528, 11122257, 20362557, 22656295, 20296923, 10400962, 1225332, 1225214, 22393189, 11120673, 936030013, 11120291, 14593621, 2665601, 15838314, 19835830, 22390717, 19834788, 19833780, 24027568, 23174555, 23305060, 19831548, 24287818, 11311295, 15964172, 24287226, 25335155, 21271810, 10785950, 10785904, 22187857, 19828476, 19828286, 22187563, 22187453, 23301558, 24415604, 10718062, 942050089, 23102684, 16745285, 24019675, 24019329, 1671499, 20348689, 23952798, 17136654, 23231173, 17594781, 22444130, 19887369, 22770768, 10712076, 10776954, 12087585, 20803505, 5860873, 13069810, 23096584, 13069174, 13068972, 24209897, 8415671, 20801907, 6317991, 20866692, 20603742, 11362651, 16933074, 17785033, 90096, 17784701, 24075831, 24992359, 4413948, 21322071, 945051018, 22762446, 24073083, 20664998, 1986418, 22826546, 22825214, 19876050, 19810245, 19809961, 19875290, 6177676, 11682693, 11158177, 20529679, 20529425, 2179299, 948060094, 22232600, 8403919, 11155693, 22624224, 19871674, 20526375, 8402161, 10695847, 20853845, 5846016, 22819576, 21901787, 11284790, 20852843, 19804229, 20852073, 20852071, 21376130, 10234587, 20261495, 11282852, 21374938, 21309239, 1188973, 9446491, 21373884, 24060813, 6168854, 18292866, 20258707, 2236043, 11279792, 24059141, 17635916, 14751664, 21435975, 23860719, 1185161, 22680860, 11210444, 17763959, 17763681, 23989231, 17762837, 17762497, 17762405, 20318172, 9373612, 20318070, 17761733, 14746878, 22676714, 22676664, 11338199, 20381109, 7732371, 2685942, 17692917, 3340116, 1373826, 8189197, 12055728, 1897164, 11332923, 9497324, 20310617, 10480049, 11397310, 7071708, 9627045, 17163230, 10674774, 10674670, 23978364, 10674486, 14802921, 945020485, 948100623, 16768454, 16767914, 16767320, 11393074, 2676736, 23844585, 14669369, 20370157, 11850219, 10408330, 23252073, 18401603, 20105363, 11323128, 11126507, 11715511, 18072380, 4308829, 19905824, 17808494, 4766818, 22592293, 948091101, 948090888, 21411751, 20297473, 948090485, 10465999, 20427386, 22393401, 3060056, 1131000020, 8957968, 20229501, 25079152, 933080194, 20228315, 23701379, 19834652, 19833852, 166174904, 9411578, 15047463, 24286818, 946050104, 17142636, 20288144, 10719384, 9998072, 3575400, 20155595, 10718394, 10718200, 14584305, 947030140, 10520334, 6129327, 6129047, 15107431, 24543777, 943030423, 10715144, 24018735, 948010211, 24017685, 20216360, 2456076, 19888355, 23361737, 20215532, 21263879, 13071692, 23033010, 5860877, 23096494, 17591302, 11364459, 16867949, 22635110, 12476928, 11362727, 11362683, 17784751, 24534516, 9984467, 8935260, 5330721, 8410701, 20665754, 1987394, 11227534, 22826912, 20467182, 3624339, 19876962, 1985140, 1984818, 22628618, 22825154, 84150, 22628490, 1984538, 19809611, 11682743, 3359544, 2179343, 2179283, 948060366, 22102031, 22625274, 8403941, 8403539, 12072778, 10696051, 22820084, 20526303, 16594089, 23015508, 23015506, 20851851, 23276198, 20850955, 16918744, 8202185, 21373848, 10232287, 946020776, 22880657, 1187973, 20848645, 18292624, 946020050, 24452561, 9903491, 20585260, 948050078, 11081793, 23990741, 4264252, 23990553, 14749976, 11211014, 11210546, 22678202, 21760652, 27724319, 10750363, 7866726, 11339595, 11339183, 17171566, 19858042, 942080036, 11337703, 20512512, 24575392, 24575380, 9305366, 11205902, 20380867, 19856434, 19856034, 8190483, 11336107, 11335709, 19855144, 19854990, 11138380, 9499892, 12055758, 935130029, 20640646, 19854108, 1176168, 20312221, 7924597, 20310663, 10742164, 5302333, 11593681, 947120033, 11396914, 10479365, 11395808, 11854359, 17162238, 20635552, 11394930, 21225259, 11984581, 18275893, 945020421, 23715228, 20831604, 23059694, 23846123, 23846031, 11393986, 14277528, 11393656, 1169808, 23845257, 931060007, 11393232, 16766978, 23058420, 8378219, 1692984, 21746869, 20829072, 19780211, 23253529, 20828126, 20827938, 11390466, 943050187, 943050182, 948030249, 5884499, 948030014, 948030010, 23513318, 23513254, 2803849, 22071114, 3917480, 11846141, 11124927, 11386835, 11386108, 4766914, 9485166, 24885880, 22591997, 11122917, 17152186, 22657097, 948090732, 22656629, 11711981, 24360063, 24490844, 20623998, 1225272, 936030067, 18329204, 2666027, 5876959, 15969429, 24357927, 5876191, 19835064, 19835062, 20621286, 20359134, 19834542, 24028774, 1221986, 1221978, 19834004, 8954960, 10003480, 23241630, 19833736, 19833620, 19833610, 23241370, 7053767, 24027338, 1220754, 1150001112, 24026978, 2662085, 23764649, 166173479, 10328645, 11311555, 3971218, 15964060, 10721064, 24286792, 23303372, 9999516, 24155208, 23499321, 10719772, 11112851, 21270774, 10719382, 4558630, 11177597, 22187473, 23104762, 20286552, 10718160, 166758920, 2787845, 12814691, 20350837, 4556562, 6129313, 1671793, 10715486, 23101646, 943030272, 17596225, 10780058, 23952634, 23952626, 17595467, 23230699, 21264379, 19887265, 11170493, 20541840, 19886479, 11432071, 20803655, 23228432, 12087207, 19885857, 12087063, 12086997, 22769088, 21261665, 21261609, 12086417, 13069232, 12085849, 11364107, 22636000, 22635960, 11363639, 24339368, 20603692, 11362735, 16933148, 11362531, 11362407, 89574, 24075721, 24534348, 1203510, 1203448, 24992881, 16865375, 24074045, 7952038, 24073693, 24073505, 20665490, 11359121, 11228016, 24073065, 16863761, 20664736, 8409049, 1986466, 1986364, 20204890, 19876924, 19876102, 21907615, 1984564, 19875620, 1984144, 19088953, 6178222, 20268441, 10700185, 4604822, 2245147, 19808775, 19808737, 21250366, 11878626, 22822750, 6241983, 3554792, 22625258, 10696977, 5846380, 19804857, 11284646, 19804019, 15871435, 19803357, 23997318, 11283274, 945040045, 8202849, 20261041, 10692495, 21309323, 21309273, 22882079, 21374460, 26748103, 15934509, 24060835, 20849529, 9904793, 17703477, 9445893, 9445741, 16916816, 24059539, 18292208, 27139530, 11279652, 11082389, 4004018, 21435919, 24057357, 22680376, 21304081, 21303865, 14750080, 14750058, 14750046, 21762218, 21303369, 11079435, 16977476, 11209840, 23268308, 17762485, 11602009, 17762369, 17172512, 14485080, 8193565, 11207844, 7865474, 17171546, 945030140, 17171382, 20382119, 10682506, 24575400, 9305316, 8190551, 10680782, 3340530, 8190105, 11335541, 1373870, 19854064, 2683504, 11137572, 9302479, 2683206, 10678504, 2683028, 2683002, 10678332, 24440698, 9629207, 19852186, 10742156, 944040180, 9496736, 22603711, 7071666, 20834128, 9496308, 19785419, 16770676, 11331127, 8315803, 20635894, 16768486, 10739087, 2677342, 2676870, 11851641, 17684130, 11851085, 11392264, 10409000, 18076514, 25088403, 17682536, 17682294, 11849189, 937020420, 948030304, 11324058, 11389219, 22071134, 11323154, 17155678, 11387606, 11321812, 5357876, 14598199, 11124763, 22069018, 17743537, 23706642, 946060054, 11123303, 1357934, 24885175, 5878899, 10466053, 21410126, 11120659, 8957962, 11120585, 4763214, 24358391, 15837708, 9349579, 22390935, 23701371, 2663847, 10003492, 11313239, 10395691, 23175199, 25075737, 23175013, 3972634, 23240078, 2858183, 19831784, 11311581, 10000842, 7902375, 4035710, 10719818, 12685767, 948080320, 12618868, 23104470, 6130357, 1411765, 12421641, 11438321, 10520366, 6129183, 6129091, 10716250, 1212755, 1409327, 10714536, 10714436, 17136860, 20151359, 24804356, 23231071, 20216266, 17594741, 15889840, 12088627, 24081685, 12088053, 12087365, 23228348, 23227886, 9989407, 17591298, 13069002, 16934900, 20867042, 948070203, 22636256, 9987729, 9987611, 6317527, 14181548, 16867809, 22634966, 11362717, 17785149, 11690134, 286751, 25059254, 89476, 89178, 9984263, 20535354, 8410603, 24073297, 11227886, 20664962, 20664704, 24989665, 20204900, 20204878, 7097233, 22825790, 19810645, 19810087, 20465050, 22824254, 24002905, 12730711, 2179289, 20594846, 948060134, 22101967, 22822462, 10960144, 9452681, 8403973, 24001227, 19806721, 11155329, 21902995, 800221, 8074646, 20853889, 20853877, 11285610, 20393383, 9448849, 10693353, 666160, 16918634, 21375052, 10692449, 16852164, 24061037, 20259687, 10232393, 10232359, 10232325, 1188311, 10690927, 20848889, 18292962, 11279860, 11082521, 948050045, 22681176, 10753255, 10753019, 22680364, 265971, 17174050, 23989247, 19860088, 10750473, 21760208, 14485032, 22676698, 19858458, 14746596, 19858172, 23134445, 12582801, 11206498, 7733079, 20512410, 7732677, 17694065, 17890660, 11205910, 9305314, 8191147, 20380513, 14678416, 3340584, 3340552, 24312069, 3340510, 3340506, 20641970, 19855052, 11334989, 9499922, 8385663, 16970856, 11138108, 166196261, 11203602, 20640660, 11334481, 8188693, 1897170, 19854032, 9499274, 2683512, 2683482, 19853890, 11203122, 10482023, 11202726, 20639752, 10677852, 11398520, 24505554, 24505514, 11398240, 11332649, 11332319, 1174106, 10742304, 11332011, 11397094, 22996919, 22603697, 11331361, 9496310, 11396788, 11331191, 10675656, 10675500, 11986131, 16770040, 21423018, 19784341, 11854103, 21422278, 1433451, 10739555, 16768854, 10739093, 23715108, 948100354, 20831468, 23845911, 23845889, 15915689, 20830486, 11393214, 23844943, 1693478, 10409744, 1693288, 16766542, 11392316, 2544629, 11391833, 21746435, 18076370, 8376889, 7197217, 19779941, 11849821, 20828138, 17682328, 17682228, 24235620, 20827630, 23186421, 948030257, 21744399, 23513776, 11323178, 11847455, 11126105, 11125951, 23839630, 14598859, 23773755, 15450140, 20365193, 20365177, 24559277, 7257691, 14597409, 19905684, 19905612, 15842198, 946060014, 25082560, 20363901, 17807988, 11123029, 19052796, 23639994, 948090486, 22656349, 20624732, 20362553, 9483518, 22655431, 22655353, 6730001, 936030004, 19836224, 19835986, 8956928, 19835652, 22653257, 22653151, 25208607, 19834620, 15836799, 932030044, 24028726, 15181205, 19834242, 10003416, 24027988, 23175763, 24027608, 23175597, 10330159, 16097069, 24026600, 1875211, 24288078, 17799672, 10000860, 19831250, 11311319, 23238526, 11310947, 19830598, 10327563, 9999178, 20288166, 21271170, 11440753, 4165974, 23302438, 11178269, 20156441, 6262654, 6131419, 11177521, 22187427, 14519517, 9997328, 23104440, 1214685, 11175933, 23234453, 167806456, 15893992, 18514796, 24019723, 24019697, 23953921, 10715514, 23101644, 2785095, 17596159, 6782259, 17595447, 10320934, 11566020, 17595293, 23231131, 21199356, 5863885, 23886144, 23886126, 11565268, 6322363, 11171949, 23361311, 23361225, 21067118, 12088265, 11432881, 11432069, 13070346, 13070334, 19886011, 12087131, 23227874, 5205444, 5860729, 12086403, 13069442, 13069108, 20802067, 9988327, 948070472, 12085173, 12085159, 92046, 6317793, 17589812, 17589810, 10773840, 22635684, 20603774, 20865740, 21258759, 11362583, 23224522, 90206, 11689840, 16932634, 11361813, 89594, 24075723, 24534386, 20536544, 9985185, 23157732, 20667080, 22633114, 9984535, 24270970, 20273209, 24074205, 1726271, 11359815, 8410659, 22762986, 24270182, 940070044, 24073213, 22631412, 1987478, 20665206, 1987438, 1987412, 11359047, 11227872, 24400054, 1200230, 12341092, 1986384, 1986350, 20205246, 20205228, 21319177, 20467158, 478545, 19876636, 14961421, 12667452, 1984708, 1984666, 22628348, 2901979, 20596572, 9455307, 19875178, 19875034, 21906623, 19874904, 20202434, 19874570, 18301522, 22823316, 21250450, 20267379, 11157829, 11157795, 2179125, 22626150, 22822574, 23871125, 24001213, 22821430, 801315, 3160499, 22820848, 19805983, 20854411, 20657424, 10695937, 20657314, 24851516, 21902159, 933180177, 1978700, 19804267, 10235941, 23015366, 20852637, 5844810, 17706201, 23997632, 23997268, 11152181, 23276312, 8203015, 17705275, 21375102, 1189943, 24193056, 17704913, 8202187, 6170546, 21374626, 21374506, 21374434, 10233199, 7087377, 11674881, 16852110, 21374058, 21374040, 24060923, 336889, 17703513, 20259405, 11280696, 18882838, 933110027, 20259051, 21241862, 16916358, 3153775, 24125285, 1187395, 11279902, 20585774, 21437657, 18291916, 8919945, 11082613, 18291514, 18291500, 17636124, 16849476, 9509162, 24057869, 24057647, 22681240, 14685638, 11277640, 22680708, 21762904, 20517569, 23990833, 17174944, 14750018, 14945924, 14945878, 17763875, 16977414, 17370338, 23989405, 17370236, 17763443, 23989319, 23989103, 17763169, 19859792, 11602011, 17762285, 11339631, 17762055, 17761907, 17761777, 17761729, 22676690, 945030347, 19006329, 19858284, 8192735, 19857970, 8192531, 23920454, 17890958, 941030343, 20380511, 10681062, 23198342, 11467380, 19855880, 17168796, 17889684, 17692819, 17495637, 1439271, 9500108, 23720969, 11203412, 9629797, 2682856, 24440666, 14806725, 1174304, 944040016, 22997031, 10807333, 11331591, 10479429, 9496384, 16770702, 2680356, 22603089, 24044691, 15918045, 11396020, 11854025, 20635760, 20635636, 24043435, 24960704, 11394738, 945020429, 2678284, 20635032, 20372803, 945020198, 945020197, 948100352, 11394102, 11852819, 17684086, 11392122, 10409074, 17159228, 2544185, 20828302, 17682574, 11390344, 948030259, 11323300, 11323180, 945080058, 9356750, 17155506, 18072978, 24036682, 20366323, 5882203, 17744011, 10272620, 17808882, 19905722, 9485200, 14596983, 24296298, 15907669, 946060050, 15448518, 10402031, 948091039, 19904412, 11122329, 24491642, 22656593, 22656321, 9483538, 20624606, 1225734, 4895009, 2666357, 15838454, 20229085, 5876583, 2665121, 2665043, 22390823, 10004298, 19834622, 19834380, 19834326, 2663195, 24027792, 14786949, 23241052, 23175461, 23437523, 10330081, 10854235, 10854129, 2662059, 11312605, 24288192, 24288036, 23239412, 24287296, 11311103, 10327271, 11441281, 10720362, 10720292, 15962944, 17469871, 6721949, 24416178, 6721335, 6131363, 19828386, 6720907, 20155047, 166758913, 4032804, 10389774, 166758475, 11438059, 6129301, 6129293, 10716564, 3572743, 7898009, 10715894, 10519268, 1671823, 1671795, 23954011, 23233093, 10518982, 23101694, 24018753, 1736282, 948010241, 948010212, 24935640, 1670357, 10320994, 11566012, 23886758, 1669921, 19888477, 24344817, 816957, 15889786, 20477238, 21328965, 21328931, 12088277, 11301774, 21328733, 12087949, 11432181, 11431903, 21261745, 5860637, 946040064, 21261557, 20802741, 22899790, 20540424, 18967396, 12085881, 23881882, 17590246, 23881546, 17589832, 6317467, 22635806, 11363079, 11361923, 89496, 20536290, 20666818, 16931212, 943020328, 7952414, 5330519, 24073669, 1201316, 16864347, 11228250, 24073143, 11358993, 24990442, 20664994, 11227600, 1985206, 8931820, 1984966, 22825354, 1984890, 1984784, 83704, 9455295, 9455293, 946030188, 21251400, 22824106, 4604868, 3162801, 22823368, 20595140, 21905847, 940130560, 7291104, 1523679, 15155144, 20266581, 19807027, 940260081, 22821358, 7879114, 22624610, 22624592, 19806347, 19806117, 21902961, 10696201, 8074744, 10696139, 20853941, 24851520, 11285562, 10695697, 11678731, 21901891, 21901881, 10695071, 20590870, 10694691, 20590602, 20392743, 11283098, 11152021, 21375458, 19802513, 17705161, 23209796, 6170332, 16918116, 21374452, 11281692, 9445999, 19865742, 2826240, 3153789, 946020003, 938090142, 9902573, 21240246, 9509248, 16849112, 24778725, 1185197, 24515952, 22812013, 21762912, 7082694, 1314997, 11079381, 23989953, 17173950, 11209952, 11209922, 11340237, 19859686, 22677320, 11601689, 9438933, 14746888, 14746868, 17171644, 17171302, 14746384, 19858078, 11338285, 11337911, 12058618, 23920468, 11206384, 19856634, 19856494, 10026014, 2161574, 19855962, 19855930, 3340794, 14678498, 19855772, 10680626, 10680610, 3340566, 13891828, 22673446, 20379679, 22673386, 946010074, 14415727, 11335517, 24311527, 10679724, 9500000, 11138376, 9499968, 9499946, 9499940, 17429694, 17495193, 11334709, 10482659, 20640658, 19854180, 2683730, 8188725, 21754632, 19854084, 25227996, 11334313, 9499220, 9499176, 20640206, 10678584, 2683140, 11202810, 10678522, 2683114, 10678500, 9629693, 16052090, 96988956, 9629309, 20311523, 24505642, 20245744, 16051220, 10349490, 20310811, 1174016, 20638204, 11332031, 9496702, 5302245, 7071696, 8316733, 12969788, 11396924, 20571637, 21095547, 17687535, 9888572, 21422878, 20636416, 10674854, 10674802, 22602315, 11854335, 26927610, 10674640, 11854049, 10477663, 20832324, 2547737, 24043297, 24042981, 23059708, 11394300, 1563892, 22142102, 11852777, 940170017, 1170136, 17160812, 23910955, 1169946, 11852025, 11851991, 11851785, 11851659, 10409802, 1693176, 23909563, 1692798, 10736765, 1561634, 2544365, 2544283, 23188055, 11850301, 20828554, 2543877, 2543873, 1691904, 11849853, 20369339, 19058530, 11390774, 11849337, 11849219, 10407334, 21744723, 11324196, 19647094, 18401585, 3852177, 22595305, 11126505, 23774857, 23774733, 7259477, 24560715, 24560695, 20300811, 20300775, 20300741, 20693936, 5882767, 24036214, 18072418, 11125559, 15844124, 17744383, 11125155, 11124997, 23838898, 11124891, 20365127, 24559415, 17808934, 19906038, 11386076, 24558735, 22592599, 2538565, 22592559, 22592549, 17742785, 9485210, 17742737, 17283975, 17742705, 24492866, 21412557, 14596725, 14596619, 14596585, 3520797, 10402051, 20428984, 21412017, 948091004, 948090987, 948090874, 19904414, 11646859, 22656711, 15447576, 6141328, 20624730, 20624640, 20624616, 9483492, 2929825, 10466245, 1225666, 20427622, 10400528, 931050029, 25211339, 22916872, 21409538, 22916552, 15838552, 20229431, 23506214, 15838452, 18328816, 8367294, 15838396, 25209934, 22391853, 17607723, 17607717, 17607713, 22653971, 19835838, 9350075, 22391711, 11119317, 11119313, 25078356, 22653515, 22391177, 19834670, 22783710, 19834584, 19244675, 11445877, 21276270, 24028504, 19834144, 19834102, 19899537, 23176299, 23176257, 19833912, 10396551, 24028000, 24027520, 24027388, 24027342, 11313305, 24027284, 10002572, 24027244, 24027230, 10854263, 11313001, 20553565, 2662229, 24026834, 2662053, 3972678, 2661951, 24026550, 1150000439, 19831940, 11312149, 23305058, 24287928, 10328559, 11311349, 10328285, 11311205, 11310929, 3970836, 10721014, 19830252, 3970518, 19895711, 11113797, 9999538, 9999378, 4559838, 10785744, 19829602, 19829536, 23499527, 21861086, 6263206, 19828900, 22187867, 22384399, 909020883, 21270168, 19828348, 11177557, 11177555, 10784228, 3575192, 10718616, 4557358, 2787713, 11110727, 11110721, 11438263, 20351113, 4556934, 166758476, 11175959, 11438051, 6719395, 23234439, 20546962, 24478931, 22905835, 23954117, 1212779, 24019121, 20742258, 943030374, 19890057, 23100800, 10714488, 21527890, 11172933, 17595453, 17595449, 17595445, 11172853, 11172847, 17595359, 10320840, 11566022, 11172805, 11303858, 24542057, 11303714, 1669923, 17594825, 5863857, 17594767, 20478032, 19887929, 20215538, 24081857, 24081801, 14710114, 942110034, 20542782, 19887297, 1406023, 20542328, 11170473, 12087869, 20541722, 21328097, 11432053, 23228462, 11431901, 21327833, 12087115, 23228194, 21261687, 24014161, 21851401, 24014073, 21261503, 23096482, 22899784, 13069378, 17591308, 5204982, 14838669, 20867798, 15034855, 20867488, 11364427, 92100, 11364243, 948070277, 17590130, 12085105, 948070205, 20866798, 21194448, 20276871, 20866672, 24340074, 17589808, 17589806, 20604208, 24339646, 20341857, 20603738, 4940565, 20865714, 22635050, 17785185, 17785035, 16605243, 22634498, 24534912, 89922, 21388982, 23027282, 89364, 1203452, 21322989, 9985191, 9985149, 11688830, 20536110, 9984481, 4413836, 23091579, 23091463, 23156758, 2119602, 24074117, 24073999, 22631342, 1200862, 5329589, 20664916, 20664808, 20664760, 1200160, 17715190, 1986378, 1986362, 1986354, 1986326, 1986316, 20205270, 20598438, 22826490, 3624227, 16927800, 19876620, 8932096, 22825720, 8931944, 1985076, 84372, 22628600, 22825102, 1984612, 12666940, 1198070, 22628334, 20465618, 24397748, 19810165, 22824798, 19875672, 19810089, 22824476, 19809805, 214475, 20596046, 20464914, 9454799, 21906371, 19874626, 10961558, 15876757, 11158137, 11157887, 19808483, 948060315, 2179213, 11157485, 2178743, 11878022, 19807865, 22822352, 15154604, 24001453, 21248882, 19807079, 22821504, 22821476, 801325, 22821208, 22624494, 11155393, 21247846, 20854459, 20854391, 8074862, 8074824, 5453268, 800165, 20854151, 10696057, 8074610, 19805527, 19805513, 8402175, 799875, 11678809, 21902423, 20853811, 11678769, 20853719, 11678617, 19804813, 5845496, 20525447, 22819196, 11284754, 12398862, 11284684, 20656110, 20393865, 11677303, 11283912, 22818164, 22818156, 15871319, 20393285, 17706097, 20851685, 23997308, 19802793, 23866014, 21375616, 11283002, 16918856, 16918826, 21375198, 4073640, 22816910, 21375070, 22226874, 6170544, 10692367, 23471856, 21374668, 22226632, 11150967, 10233295, 9446851, 20391319, 21374342, 15934853, 11674855, 11281402, 20259689, 18293494, 1188339, 21111276, 8200535, 9445623, 6168812, 1187999, 20848769, 5251183, 9379944, 7151336, 24452823, 946020046, 1187473, 21372554, 2236023, 11279936, 18292280, 24059261, 18291918, 21371948, 24386573, 4004566, 11082413, 17635844, 9902367, 948050179, 948050161, 1186021, 24058075, 948050081, 22681748, 948050063, 8460380, 20715560, 24057845, 21436325, 661300, 15865643, 22681130, 22680924, 22811979, 14685354, 9442423, 9376700, 22680470, 21762876, 9769786, 22680340, 23990815, 23269620, 6688983, 21303341, 23990315, 2297880, 23990253, 20451300, 14946166, 21303089, 266007, 17764105, 265991, 11210486, 14945790, 17763801, 11341019, 11209862, 11209786, 23989301, 12586001, 23989041, 22678314, 23989021, 22678200, 17762833, 23333268, 17762483, 24578129, 24578113, 17762317, 3999748, 20318208, 17762301, 14747412, 15861511, 19859108, 17761783, 11339247, 13632919, 17761573, 9503922, 17171606, 22676598, 8913909, 17040301, 10683112, 11338365, 5308770, 942080032, 7733085, 7732405, 19856546, 3341244, 19856148, 24115987, 22673782, 3340496, 23066799, 16054418, 23066663, 23066403, 2684284, 9499840, 9499834, 8385669, 8189059, 8188971, 935130031, 8188719, 19854068, 2683478, 20639700, 2551689, 9498452, 10678060, 9629321, 10677880, 1174722, 11332757, 7269321, 1174228, 944040185, 944040175, 11331791, 11331763, 9496664, 11593635, 11331399, 9496050, 11396592, 17687875, 11396326, 5301133, 9626359, 11395690, 20635824, 11198368, 11394556, 23715232, 948100359, 26270231, 1169870, 11851871, 2676670, 22599597, 17684132, 17684088, 11851181, 21747107, 11850233, 11391372, 11391360, 8376659, 10407496, 20368860, 23514002, 948030006, 7915821, 23513336, 24037470, 22071108, 11323166, 11323132, 11323114, 23512738, 15713817, 23774731, 11322698, 20300797, 18072500, 22070182, 18072396, 11321860, 11321846, 22069726, 17743969, 22069072, 19906312, 17743451, 17743379, 14597437, 16039176, 19905644, 19905632, 9485238, 22592425, 22592389, 17152762, 14596721, 24295968, 14596613, 10402239, 1226878, 19904416, 15841170, 18331378, 5878903, 6140978, 20427678, 10466147, 11121445, 10466015, 17150610, 931050024, 11121175, 21410132, 14594327, 18329460, 936030035, 8367908, 1224192, 22392051, 15838350, 18328600, 18328518, 5875859, 25208974, 17541149, 12691418, 23701429, 25077396, 18523719, 24028636, 10003732, 19833796, 10396251, 10395929, 23175393, 11313295, 11313161, 2662213, 23174957, 11312761, 16817525, 23894394, 11311477, 11311185, 11311159, 11311129, 10786320, 9999754, 11113769, 10327223, 6722649, 10720284, 946050064, 19829648, 11178891, 23106338, 10719504, 6721761, 20156553, 11177857, 909020965, 1216259, 19828394, 6131367, 909020787, 23957018, 23104776, 2788557, 18517100, 7900123, 10718084, 942050026, 11438287, 11176041, 166758421, 1411073, 20154327, 18515698, 10520202, 10716450, 10519596, 16745247, 23954127, 23953983, 10715596, 10715484, 2785049, 10714558, 10714510, 17595443, 25852910, 17595349, 23231165, 23231125, 17594799, 17332427, 23230605, 22771788, 817115, 816441, 19887255, 12088259, 24604493, 17657295, 5860735, 21261523, 21261507, 5204702, 16934888, 22636304, 91732, 5203218, 24339568, 17785151, 16933104, 11362373, 17391431, 1990244, 89608, 89484, 9985225, 9985223, 1203388, 89272, 20667022, 22633000, 16931346, 16931308, 16931180, 24271134, 8935388, 11360161, 5330535, 16864753, 20665832, 23877037, 24073629, 8410523, 16078152, 4150574, 1201194, 22631324, 1987424, 1987388, 11358979, 945050227, 16863723, 8409207, 17715200, 16928616, 1986386, 1986342, 16731710, 20467194, 478401, 20597680, 8932218, 18303848, 18303846, 1985318, 22825704, 1985200, 1985126, 1985084, 24201681, 8407501, 22825336, 19810655, 22825288, 1984668, 1984618, 946030200, 7947870, 22824494, 83208, 19809507, 9454811, 21906393, 15876907, 19809035, 2179707, 22823468, 2245139, 19808733, 3359076, 21250382, 7291114, 2244831, 2179189, 19873724, 11157389, 22822592, 22822420, 14826987, 20331504, 10959808, 7879530, 10959640, 19806955, 800801, 22624168, 8009565, 12662476, 10695895, 10695877, 10695771, 11285578, 4469805, 3879550, 21901829, 1978684, 7156004, 21246197, 11677329, 23997842, 4468099, 20393299, 20261501, 19802543, 16918800, 6170770, 21309225, 23209760, 26814143, 16918168, 16918142, 21374538, 11674883, 664628, 21308465, 20784012, 10232377, 17637826, 11411820, 6168830, 18292856, 9379946, 20848639, 946020047, 21306961, 11279916, 2235845, 20585370, 203663, 21436219, 14685762, 1185337, 18290194, 10753403, 10753387, 10753325, 267545, 16978990, 11277340, 22680306, 4002092, 11211018, 17371068, 11210564, 266045, 11079359, 265891, 14945860, 17173794, 22678182, 7932521, 7932483, 21760542, 11340219, 10815336, 22677324, 8193645, 14681604, 17761645, 14746882, 22807703, 17171562, 10683482, 14746666, 10683230, 23135011, 23134779, 22938092, 4915508, 7733287, 12582799, 9436877, 20512358, 11206236, 20512328, 12058162, 11206082, 12057998, 9305362, 19856558, 19856516, 11205748, 13106265, 20511704, 24116031, 17169118, 8190675, 8190571, 22673892, 19855840, 20642246, 20642184, 24115555, 10680638, 3340592, 3340574, 22673688, 10680554, 3340516, 22673618, 8190159, 3340474, 17168442, 23066667, 8189933, 17168354, 20641612, 21296955, 1373868, 9500066, 9499986, 23065907, 16971000, 11138236, 17429670, 8189079, 8385665, 8189051, 8189007, 20575253, 19854344, 20575197, 19854264, 11334549, 20640656, 12972892, 19854158, 17494787, 8188669, 9499322, 9499306, 23720595, 2683516, 2683508, 14152290, 16052762, 20640230, 2683280, 9499024, 10678598, 10678570, 9498872, 2683098, 2683070, 8384631, 7139418, 16052232, 2682872, 2682834, 11333547, 2682722, 10809161, 11333443, 11398972, 2485906, 8384143, 17165942, 18869832, 10677798, 10022400, 11333057, 17034671, 3599774, 18869600, 2682198, 20638948, 11398212, 10480615, 1174482, 5303213, 15920033, 1174334, 20310797, 9497346, 1174180, 1174174, 11332189, 11463242, 11332061, 11332055, 1173960, 10742212, 11331915, 22079793, 11397378, 4122878, 944040178, 11331813, 24045777, 21096629, 9496592, 22996987, 11331365, 11396860, 9496304, 2680502, 19785353, 17163866, 7923245, 10675694, 2680272, 20833546, 20636700, 4121508, 17687425, 8315729, 22602447, 20439708, 20832890, 11395634, 20505066, 10674536, 10674500, 20504838, 10674390, 21422272, 20635838, 20635802, 11853971, 20635782, 20832356, 11853905, 11853875, 11395022, 20635586, 20635574, 20635504, 1564512, 11853505, 23715464, 11394690, 948100730, 945020483, 23715378, 20372933, 11394498, 23715190, 23715132, 1170722, 20372739, 23846083, 20831410, 11394224, 20372633, 11852897, 22535258, 11394122, 22141954, 17619898, 23714716, 23714710, 10017622, 11393854, 15915759, 11852423, 11393586, 2677284, 20830536, 20371719, 20371709, 11393266, 8378493, 21223539, 11393092, 11851829, 10409872, 8378195, 1693522, 1693518, 2676550, 1693502, 1693490, 10737455, 10409754, 10409746, 10409700, 1693308, 11851171, 11392352, 18076950, 2544847, 23909545, 2544803, 14668959, 13292534, 2544549, 11391890, 22532870, 7262925, 22532714, 11850297, 8376869, 7262753, 19780117, 19779961, 11850087, 8376665, 10408266, 11850003, 19058948, 11849713, 1691532, 14667585, 14274276, 20827692, 11849235, 20368908, 11390258, 24235256, 24235250, 11848945, 10407052, 10406842, 7261059, 10734406, 7260969, 10734276, 2804373, 11324024, 11324010, 21744229, 9619771, 1558798, 22595803, 22071276, 22595477, 11323280, 16041864, 12699488, 20694854, 945080060, 11847315, 17155612, 11847191, 9356746, 18072996, 19580203, 18072872, 12698910, 14599441, 23839804, 24036366, 18072534, 18072512, 18072438, 18072340, 17154678, 22594097, 11321884, 23839218, 22069684, 7913906, 7913896, 947110242, 11845985, 5882169, 5619990, 11976933, 11845795, 17744003, 22593625, 11124707, 11386844, 11124673, 20365171, 20365165, 11189923, 24297058, 19906034, 19906032, 24493490, 19905770, 21085410, 11386059, 23706672, 17808420, 22658067, 15842266, 14007254, 17808316, 17742757, 24492830, 20429570, 22592141, 14596733, 14596649, 10402343, 14596633, 14596629, 22592001, 14596567, 20298077, 20298075, 17807656, 948091093, 22591681, 21084246, 22657077, 18266153, 22657043, 11188191, 948090759, 3520331, 17807154, 20297495, 21739152, 948090447, 20362731, 945010149, 2733484, 2078089, 20362613, 5879063, 20362423, 24360113, 898185, 22656089, 22524951, 9483286, 10466271, 1225678, 1225676, 1225658, 19772313, 10466191, 10400508, 19772093, 1225378, 20623964, 17150528, 931050021, 6730275, 17150476, 22655473, 17150390, 14594409, 21410116, 4763920, 8957966, 11120623, 22654853, 17149502, 22654483, 1224198, 20557287, 14593411, 15838508, 14593273, 15969487, 15838394, 20622330, 8039397, 22391737, 19835820, 15903605, 20228979, 18328358, 19835594, 2664993, 22653271, 305369, 20228301, 22653131, 23505014, 22652969, 20359108, 7972661, 2664211, 19244623, 2664005, 19244597, 19244525, 24028590, 23242138, 1221992, 1221922, 19834100, 11314345, 8954962, 19833932, 24028202, 23700517, 14918655, 10003376, 19833752, 23175977, 23175907, 23437973, 18850286, 20226525, 22454731, 19898757, 3252587, 24027288, 24027270, 11313229, 10002472, 10395681, 10395657, 22978544, 10854249, 3972932, 19832642, 10854105, 10854049, 19832464, 23240318, 18325088, 3972664, 24026642, 24026602, 166174093, 7969949, 23239680, 19897205, 24287878, 24287872, 17799776, 10328619, 10328599, 10328581, 24287702, 25205200, 25205198, 24287690, 19896775, 10328487, 11311515, 24287594, 24287544, 11311393, 11311243, 22976610, 11311195, 11311163, 11311133, 4560864, 11311029, 11311019, 20223817, 10721052, 3970756, 9999816, 9999796, 9999752, 14521607, 2724909, 19043249, 9999158, 4559610, 23302808, 11440713, 10719816, 10785336, 10719730, 17142212, 13210028, 4034958, 23499099, 10719570, 21270782, 11112511, 6721589, 24416200, 11177775, 11177685, 6131381, 948080305, 11177625, 9997964, 19828302, 3575372, 23104840, 23104830, 6261934, 23104650, 21269634, 21269632, 1215533, 18516924, 10718126, 10718002, 23104260, 942050016, 11111037, 20417134, 3836437, 11700696, 3574212, 2787693, 23234759, 14518467, 11438265, 18516142, 1214569, 20351075, 18515936, 11438047, 11175791, 10520398, 11175741, 10389242, 20547300, 23430876, 17335989, 17663635, 23234113, 5866939, 20546848, 20546702, 10716256, 1999928, 20808743, 10716194, 10519578, 5866471, 10716058, 3572605, 3572599, 1409767, 23954053, 1213019, 1212997, 23953963, 23953937, 10715552, 10715482, 166756387, 23101364, 19890063, 20348573, 11173379, 11173361, 11173345, 11173309, 11173235, 20741476, 1670355, 23887058, 11172959, 17136730, 17595455, 24804376, 11172859, 11566038, 10320838, 1342366, 2456130, 17594831, 20216244, 17594803, 17594797, 17594777, 17594749, 20216180, 23886180, 21068090, 6322231, 940210215, 13072060, 20215420, 13071890, 11171341, 19625472, 21263847, 21263823, 20477374, 24081747, 15889742, 19887373, 19887363, 19625176, 17593327, 23950205, 10777468, 12087927, 17658463, 12087681, 10776958, 20541784, 22901076, 12087587, 12087579, 11432147, 11432131, 11432119, 3109021, 18313334, 12087381, 11432013, 11431961, 12087145, 12087087, 6778537, 17657423, 12086839, 22769156, 5860865, 19885569, 17657333, 17657329, 21261805, 23096812, 19885543, 17657317, 11759074, 22769006, 17657183, 24014155, 17657161, 18967870, 17657133, 21261459, 5860479, 5860449, 13069304, 5204980, 13069282, 5204944, 13069208, 17591158, 17591072, 20802329, 24210171, 13069006, 20802213, 13068944, 20802061, 9398586, 2648362, 15886518, 11364409, 23881752, 17590294, 17590264, 23881662, 15886178, 20211522, 22636352, 23881498, 22636310, 21194444, 18966176, 9987635, 20866610, 17589802, 17589782, 11363623, 7300250, 20603968, 22635528, 11363219, 20603784, 11363201, 18375548, 20800359, 20603730, 13067086, 11363009, 22635120, 4416004, 22634998, 11362803, 21258641, 21258629, 21258475, 21258469, 16933022, 2646074, 14639159, 17457175, 17784737, 89912, 16932610, 14704378, 20864665, 20602508, 89592, 89572, 24075719, 17587628, 24534372, 8412503, 1203488, 1203390, 1662063, 9985101, 89158, 11361169, 2644880, 20536134, 17586954, 17586892, 11688566, 22632966, 16931302, 20666844, 4413832, 943020348, 16931110, 22632646, 22632612, 24271008, 11360381, 22632380, 11360145, 11360121, 4085592, 11359915, 11359811, 11359787, 24073707, 10376604, 6640886, 20665580, 22631594, 17716302, 11097159, 17716282, 20206574, 24073177, 20206498, 11359089, 11359007, 20665060, 11358945, 4412124, 13652458, 20664666, 19091717, 1986572, 16863243, 20402081, 22826910, 1986420, 1986366, 3952411, 20467408, 17714886, 22826564, 20794818, 478533, 22826264, 11160653, 11160561, 1985276, 24201789, 1985068, 1985036, 1985018, 8931616, 19810591, 8931578, 22628596, 8931520, 22825104, 22628494, 19875968, 20465762, 19875926, 22628436, 20465734, 22628346, 22628302, 22628296, 9455557, 20465600, 19810221, 20465470, 19875630, 14042911, 14042909, 2180763, 19875476, 1984044, 22824464, 20596208, 2180585, 19875292, 22824350, 19875170, 19809603, 19875124, 19875106, 3359966, 6177972, 19874970, 21906583, 2180153, 18301962, 4604842, 6177668, 6177644, 3359370, 22561414, 15876687, 15876675, 15876669, 15876667, 3162645, 21250532, 14434673, 4604264, 12730719, 20529477, 3555564, 21250216, 2179191, 2179081, 1523673, 22822846, 948060080, 15155090, 20463444, 23871309, 12664648, 22626106, 22822708, 22822698, 22822684, 22560520, 11878088, 22822552, 22822528, 22822454, 22822438, 17448321, 10960120, 19807389, 19807275, 24001517, 21248872, 10697577, 11287394, 11287356, 19806909, 4602510, 22624836, 801305, 11155913, 19806637, 19872172, 19872130, 19872088, 10696991, 19872000, 800815, 16594857, 19806101, 23476032, 19806007, 19805991, 4142568, 19805529, 20591932, 19805495, 5453058, 22820066, 10695899, 11285588, 20394999, 11678673, 7156596, 8205171, 3879582, 9450141, 3879530, 23015948, 11284846, 20853071, 5845282, 10367208, 11284710, 20852951, 21901497, 11284662, 20852909, 20852891, 9449587, 20394061, 9449549, 19804203, 11677737, 20852751, 19804077, 19803965, 20524849, 20524845, 22818592, 22818590, 21376788, 11677459, 10038974, 24194690, 16592435, 24128991, 20393427, 20393409, 18885960, 19803451, 22818092, 11676969, 19803409, 11283730, 22818040, 23997654, 10169492, 21376134, 10169464, 23866486, 18295884, 8924203, 6105963, 9448195, 23866098, 11152107, 8202969, 10693331, 11152061, 21375546, 10234383, 11151859, 15739357, 17705417, 11676069, 11676057, 19802493, 17705323, 16918868, 8202519, 16918776, 8202449, 17705139, 23865516, 17705109, 16918658, 1189955, 24979378, 21374844, 21374810, 7218960, 21374724, 21374688, 10167990, 21308991, 1386022, 206111, 11674875, 8922335, 11674833, 16852166, 8922187, 20849641, 9446367, 8921935, 8201035, 16851690, 9445907, 24060401, 24322528, 9773497, 15933851, 9445671, 18882812, 20848855, 18292882, 9445471, 18948141, 20258715, 22814597, 1515347, 938090258, 946020049, 2236029, 21437969, 11083219, 18291936, 1187005, 1186959, 20847745, 11279472, 24452129, 21371900, 24452041, 8919959, 20716422, 24386291, 21305993, 11082369, 21437037, 17635850, 21240292, 20715878, 9509194, 16849104, 24057957, 23861341, 23861259, 21436281, 15865633, 1185303, 23860733, 1185249, 22812107, 7345603, 14685632, 21435825, 10753381, 22680906, 22811965, 18289944, 22680782, 11277462, 10753103, 21304349, 1184531, 21303935, 23990899, 21434835, 23335318, 21434761, 16978296, 14750060, 21303629, 21303623, 5312802, 23990329, 13635601, 16977934, 23990263, 11210562, 21303103, 23990063, 21303047, 17763981, 17763961, 17763941, 23989831, 17763853, 16977394, 11210202, 17173978, 4001098, 17763629, 12061912, 23989441, 11209818, 17370174, 23989199, 22547405, 23268288, 23989145, 23989133, 17763171, 22678162, 22678120, 22678054, 17762791, 17762769, 17762765, 22677872, 21760312, 20318466, 21760250, 21760186, 17762465, 17172628, 17762329, 11601795, 11339647, 10749815, 11601707, 15861509, 9438959, 14681808, 11339205, 22939074, 11339093, 11339051, 13632631, 7865416, 21103539, 8192917, 19006315, 19858246, 11338301, 22675920, 23921062, 22675862, 20382075, 22675804, 11338067, 17170754, 7733109, 8912751, 2555703, 9436931, 19856976, 20512322, 17890710, 9305340, 19856568, 20511692, 17562558, 10681060, 17889994, 11467388, 17431024, 17430998, 16054630, 3340560, 20641658, 3340114, 15857471, 15070728, 9500144, 9500124, 19854730, 9500038, 9500018, 9499962, 11138346, 3405085, 8189159, 1176476, 17888146, 1897142, 10482259, 2683464, 2683354, 9499086, 19853744, 2683288, 2683286, 2683246, 10678624, 4387151, 10678574, 10678572, 2683052, 9629861, 8384657, 2683018, 2683006, 2682996, 2682818, 22540144, 8384119, 11333205, 24505644, 10480801, 10480535, 11397796, 11331819, 9496770, 11397092, 9496538, 7071676, 10479439, 10479409, 19785391, 9496064, 4252960, 17687817, 11396358, 17163388, 10806357, 16770036, 1237907, 17621897, 4318032, 8315545, 9495154, 9495136, 20635850, 20635534, 23977830, 11198190, 24960712, 20635222, 10477121, 2678268, 6741478, 10739165, 6741188, 17161366, 11852837, 25156361, 16963917, 11393104, 2676760, 11851775, 16766964, 16766916, 16766860, 16766826, 2676442, 2676402, 2676392, 14669367, 18077210, 14669313, 2676148, 18076970, 10409108, 21746835, 10409094, 18076436, 7262831, 7197223, 11850265, 21746189, 17813400, 16764666, 17813220, 11390556, 10407364, 2805035, 21744667, 11324018, 14600565, 20564319, 2803929, 2803791, 23971801, 11847405, 23774725, 15713795, 18073068, 14599487, 11191553, 18072620, 20300759, 18072520, 18072480, 5882763, 5882757, 24036222, 22070102, 24560265, 23839358, 7913984, 7913982, 7913980, 7913978, 7913976, 11321848, 7913974, 7913966, 11321828, 5882159, 17285263, 5881975, 14598099, 22069122, 20430596, 22068758, 17808918, 24296926, 9485652, 4766976, 9485204, 4766516, 14596887, 22788838, 20429516, 14596687, 15448526, 15448500, 17807680, 12695576, 47094, 18331572, 24491764, 11712063, 2078079, 6075719, 18330816, 6730885, 5878877, 5878875, 5878849, 5878735, 5878705, 5878699, 5878689, 5878675, 5878667, 1225586, 10466157, 10466145, 10466129, 9352003, 24424905, 17150388, 22655389, 17150316, 6730037, 24358999, 11120665, 15904703, 15183571, 14593697, 18329222, 11316731, 15838556, 25210089, 5876951, 22916162, 5876339, 22391001, 20228261, 21276742, 5875731, 22390511, 23701229, 25208293, 1221988, 15181153, 23700769, 23438355, 19833860, 23438337, 23438103, 3711754, 25207045, 17080405, 10395693, 17014709, 10330009, 19832714, 4103961, 2662067, 18521753, 25206045, 11312155, 10722302, 10001276, 25205501, 10328549, 17340785, 10328135, 4560836, 11311035, 24155512, 10720294, 17142674, 11178857, 4559602, 10719914, 10719666, 6263196, 6263186, 11964809, 21532900, 10719398, 6262922, 6721667, 20156439, 909020963, 948080393, 22187731, 22187661, 11177603, 948080196, 909020705, 10718536, 23104784, 17861850, 166759518, 7900109, 6130457, 20155099, 18385445, 20547862, 11438259, 6719385, 10520392, 4556550, 6129349, 17335999, 6129281, 23823907, 2000356, 5866619, 5866603, 5866543, 3572755, 10781682, 23430042, 2655118, 1213127, 23954115, 10715842, 23953941, 23953929, 10715602, 10715476, 20545818, 10715136, 2785107, 17727288, 17333975, 294517, 818619, 818525, 11566166, 24804388, 10320906, 10320886, 10320876, 11172813, 25852878, 10320680, 10320678, 10320500, 17594823, 17594791, 17594773, 11565292, 1407097, 11171959, 6322267, 13071886, 13071884, 24081821, 3175686, 19625164, 17659079, 19887247, 18772986, 12088231, 17593243, 12087943, 5860869, 17657145, 5860525, 2649070, 4287061, 11561372, 11561348, 24471906, 24209569, 11364231, 22636358, 17589838, 17589796, 9987567, 6317473, 19817753, 17785973, 22634984, 22634936, 936010037, 8413243, 20864941, 24534938, 17391451, 89878, 89642, 286249, 89620, 89576, 7298502, 1203616, 8281271, 8281261, 1203362, 11688724, 20666942, 22632998, 16931360, 16931338, 16931326, 16931174, 24271004, 24270888, 8935266, 22632122, 5330579, 5330465, 24073671, 8410539, 22762920, 24073633, 20206848, 23876841, 20665538, 8410283, 1201264, 22762554, 3953633, 11228100, 1987494, 15160228, 24073119, 20665048, 5329451, 17715726, 22630830, 17714910, 1199640, 16928266, 478665, 478663, 16928078, 22826288, 16927998, 478455, 478397, 478361, 3230643, 3230609, 1985314, 22825698, 1985150, 19089793, 20465832, 1984594, 9455697, 19875616, 83728, 19809875, 24004097, 1197514, 9454999, 4605202, 19809545, 3359954, 6177962, 22824098, 3425434, 20464706, 6177678, 3359498, 1983102, 4473361, 3359242, 19808571, 2179261, 21250214, 2179207, 22822760, 3358396, 2178705, 22625584, 10960150, 10960132, 10960126, 22821808, 24001273, 19806965, 22821574, 11876930, 11155871, 13121676, 1718251, 22624204, 20854609, 19806009, 11155215, 79552, 11155127, 21902947, 16987524, 10696055, 11547920, 10695931, 24851510, 11154409, 19804805, 13513204, 22819170, 1978710, 22819156, 20852823, 10235665, 14495354, 19803459, 23997636, 20851813, 22555724, 23866258, 23866220, 10234615, 10234603, 23865948, 10234377, 23865726, 16918900, 16918860, 16918764, 16918670, 8202359, 1189905, 1189903, 23209820, 16918364, 16918304, 6170338, 16918166, 16918090, 10167846, 16917996, 1713536, 664616, 9446401, 11281336, 3154645, 3154613, 3154585, 10232441, 16851576, 16851574, 16851558, 17572443, 20849169, 11411884, 1188065, 3154063, 18292854, 18292852, 1515345, 21306977, 9510058, 9444453, 18291794, 16849818, 21437059, 11082355, 13376095, 24058295, 948050147, 948050071, 948050050, 16849004, 10753879, 5707350, 1185341, 1185213, 22812091, 22812019, 14685548, 22811993, 10753351, 11277608, 10753301, 22680716, 20517831, 14947202, 25957143, 22680326, 22680314, 14750314, 4002332, 23990725, 23990691, 14160078, 23990467, 2297868, 23990199, 11210568, 16125757, 11341575, 23989893, 16977430, 17174030, 1576408, 17173956, 24186108, 11340935, 23268316, 17763157, 15862365, 19860022, 11340291, 10815910, 17762717, 10357062, 17762507, 17762501, 11601983, 25233113, 7865876, 21759308, 14746940, 14746900, 14746890, 11207800, 8192915, 8913663, 23134687, 23134621, 16056634, 25297124, 9437361, 12058604, 10682340, 11337681, 9306062, 12058546, 11206558, 4980628, 17170310, 20512616, 7733087, 20512590, 11206460, 13500188, 20512524, 9436927, 11206358, 11206298, 12713624, 20512348, 9305530, 20381113, 20381111, 12058026, 3538313, 11206026, 11206024, 13106549, 2686252, 17693973, 17431814, 20315380, 17693889, 8191135, 11205754, 7732343, 17431640, 19856466, 2161598, 3341238, 2161564, 24116121, 14482322, 2161544, 3537799, 1440625, 2685782, 11467582, 11467578, 24115977, 10681058, 14416591, 24115919, 17889972, 11467398, 11467390, 11336263, 8190463, 3340778, 20642226, 10680738, 8190353, 11336053, 8190309, 11336037, 11336009, 24312131, 8190261, 11335989, 10680606, 8190227, 8190215, 10680568, 21756142, 21756140, 18348253, 8190115, 23066787, 10680436, 10680430, 16054364, 20379719, 21755940, 21755932, 20641808, 16644100, 22083501, 20641666, 17430400, 1701740, 3340100, 15857429, 10024698, 1373864, 10024530, 3405375, 17429994, 26408400, 10679752, 24311237, 9500092, 9500082, 3405145, 16971074, 17429822, 4912436, 11334961, 9499952, 11334951, 17429774, 11334903, 8189171, 23065809, 12055760, 12055748, 17429682, 17429660, 8189071, 11334785, 8385653, 11334771, 11334715, 11138092, 16970782, 12055566, 11334667, 20575205, 11334621, 11138010, 10482617, 12972910, 12972906, 11334407, 11334391, 25228002, 1897184, 23065303, 11334341, 23720575, 2683452, 2683350, 2683342, 2683334, 2683332, 16183736, 20640004, 21754080, 2683082, 9498724, 2682968, 21753928, 2682952, 2682934, 9629707, 2682870, 16248809, 16248807, 2682830, 2682822, 9629575, 20573973, 10350350, 10677970, 10677966, 10743482, 10677944, 17297064, 10677872, 24964696, 18869836, 18869800, 9629191, 2485756, 10022312, 16051602, 17296770, 11332963, 18869598, 11332879, 16051454, 24505560, 11398354, 11332751, 17296506, 1174456, 10480531, 20179853, 10480509, 18869106, 10480419, 20310735, 20572861, 1174192, 11397806, 11332235, 10742354, 20310559, 20179433, 10742242, 20638160, 1173902, 11331951, 11331935, 1173778, 5302505, 5302499, 9496764, 4122786, 9496724, 11397202, 11397126, 9496572, 22603731, 1566674, 9496532, 8316875, 9496518, 9496508, 6743988, 22996861, 8316759, 11396950, 10479441, 22996777, 11396872, 17688297, 20506334, 20637366, 11396782, 11396780, 16770726, 11396764, 11396748, 11200084, 11200074, 2680384, 11200050, 5498412, 11331113, 20571635, 16770540, 9496038, 11331045, 10675658, 10675652, 2680244, 15852925, 22472039, 22603055, 10675492, 11396364, 17163528, 16770160, 4252736, 17556511, 20440084, 20505570, 4121498, 17687403, 21422886, 21422880, 11395766, 19784337, 10674826, 18866798, 11395600, 10674548, 21356903, 20504934, 10674534, 11395424, 10674514, 17162558, 11854125, 20504860, 20504858, 20504852, 10674448, 10674438, 11854063, 11854043, 9494742, 20635842, 20504768, 20635806, 20635756, 7266405, 20635746, 10477665, 17162338, 11984977, 20504612, 20635678, 20635610, 20635572, 1564572, 11394858, 11918989, 10477189, 948100735, 10477175, 10477153, 6676056, 10477135, 10477127, 948100675, 10477083, 10477071, 23715342, 10739163, 6741420, 10476965, 20503928, 8838470, 1563938, 1170706, 1563878, 11394210, 23846049, 11394192, 11394170, 22535284, 11852895, 6675540, 20372561, 6675532, 6609982, 24894524, 11852847, 11394066, 2677738, 3529643, 17619882, 11393868, 2677296, 19061266, 10410468, 2677194, 23910857, 20371913, 20371897, 23910801, 20371793, 2677064, 1169736, 2677058, 1169664, 23910643, 11851995, 11393236, 11851983, 11851981, 11851973, 20371637, 11393192, 3528871, 10016924, 11851903, 20371553, 11393082, 15980569, 20371459, 20371443, 15980515, 20371423, 7264217, 17160148, 3528649, 10409918, 17160122, 3528631, 3528625, 10409896, 10409890, 10409888, 10409882, 2676628, 7264137, 10409868, 23058306, 22599551, 6739838, 10409854, 3528567, 10409838, 8378209, 22599487, 8378167, 10737445, 10409760, 10409708, 10409704, 10409690, 10409658, 3528361, 17684102, 1693306, 11392582, 2676256, 18077216, 1693152, 2676190, 2676142, 11851125, 1692974, 11850991, 2544859, 2544831, 11392172, 14668965, 14668951, 2544789, 2544763, 21746809, 21746797, 10409048, 23254097, 1692722, 1692704, 10408988, 1692698, 941020120, 18076522, 11391848, 18076492, 18076482, 2544407, 20370165, 2675414, 2544321, 21746329, 19780197, 19780193, 16699963, 10408422, 19059170, 11850191, 11850177, 10277300, 19780007, 10408356, 10408314, 7262551, 11850047, 1691940, 11391238, 11849981, 19058910, 1691860, 13553821, 11325380, 20369337, 11390818, 11849481, 21745383, 11849415, 10735246, 8375937, 20368936, 20368924, 20368914, 20368910, 19779001, 11390386, 23514538, 10407292, 2805017, 17812732, 17288203, 2804733, 2804603, 3197746, 21744411, 11324186, 3853061, 11324136, 11324130, 10734302, 11324120, 17287887, 11324096, 11324074, 14600813, 7260769, 11323990, 11323984, 21744187, 14600569, 19843282, 20564139, 22071368, 4900767, 903055, 17287057, 10209145, 24561517, 11847471, 11323146, 11847331, 12699290, 23512704, 14665267, 14665263, 4900371, 11126143, 12698942, 18072890, 12698920, 18072864, 14599443, 20563191, 14599385, 21415117, 12698778, 11125881, 18072624, 20300837, 18072608, 11125791, 20300799, 20300785, 20300777, 20366299, 9356232, 18072508, 9356218, 14599067, 11125653, 11125591, 11715411, 20366127, 3851037, 22070022, 17351405, 14467801, 14467773, 22069948, 11125423, 24036002, 5882427, 22069790, 7913970, 7913964, 7913962, 24560077, 11715017, 7913918, 7913916, 7913910, 7913908, 24560001, 23839060, 24559921, 20693292, 11845891, 24559829, 17285303, 11124885, 17743995, 17743979, 17743977, 11124837, 11124827, 11124713, 20365191, 20365187, 6602546, 20365103, 2080490, 11386502, 17743471, 4767244, 24296958, 19250669, 11386330, 19643808, 9485654, 11713841, 19905834, 19905780, 19905764, 17808566, 11386028, 17808546, 24493198, 14597263, 17742987, 22658163, 22658149, 17808450, 22658109, 15842338, 4766670, 17808330, 17152970, 19643330, 17283937, 24492886, 17742661, 17152834, 22592305, 14596895, 14596855, 22592187, 16759474, 17808044, 17808022, 17808002, 24296012, 14596675, 3193410, 14596595, 15448548, 17807810, 15907217, 10402191, 14399852, 12695896, 948091194, 21084442, 10402067, 17807606, 21412009, 21084306, 1226890, 1226882, 21411953, 11122789, 24295496, 17283109, 12695556, 24295380, 21411755, 20297591, 3520373, 12695386, 1226560, 20297503, 7255831, 5617426, 20297461, 22525635, 17807034, 20297385, 20297377, 20297373, 22656647, 11712121, 19248741, 22656607, 20362805, 20362799, 20362791, 20362789, 20362783, 22656531, 20362751, 20362741, 20362713, 2077997, 20362535, 6141212, 22459673, 20362503, 20362499, 20624618, 20362455, 5878993, 5878983, 6141092, 24229015, 5878865, 15905853, 1225702, 1225692, 1225672, 1225654, 10400652, 10466155, 20624230, 5878617, 10466131, 20624202, 20427580, 1225524, 10400544, 10400520, 1225462, 10400500, 10400496, 10400450, 19772091, 1225372, 20624026, 1225348, 17150576, 20427352, 6730323, 931050028, 2929197, 20296229, 17150498, 11711005, 16101915, 1356302, 11710987, 1225226, 20427266, 24883703, 25211349, 25211335, 17150398, 17150392, 20296111, 22655383, 22655381, 22655367, 22655351, 14594421, 21410152, 17150310, 21148004, 21148002, 17150302, 7057757, 21410138, 21410134, 14594363, 17150258, 21410084, 17150242, 24424713, 18329852, 22655193, 21410008, 7254213, 20295797, 11382869, 22130673, 20295593, 22654883, 947040126, 936030072, 11710297, 11120473, 11120459, 11120453, 7253819, 15183551, 22392427, 4763234, 21409346, 14593595, 1224186, 1224162, 17608073, 19836222, 4894011, 20229429, 2665779, 8039715, 15838480, 5876949, 8039625, 15838356, 15838328, 15903857, 19835978, 2665535, 15903751, 17607613, 20229003, 9350003, 20556643, 20556623, 20556617, 20556597, 19835660, 14396044, 18328134, 5876251, 6597120, 5876217, 6597090, 18327974, 9349537, 19835180, 23177477, 22653159, 22653111, 21276852, 19835040, 23505046, 22653065, 22653061, 19834962, 19769383, 22652943, 10004424, 22456229, 21276560, 2664247, 19244839, 19834640, 15967969, 19834568, 19834564, 15902375, 2664067, 21276188, 19244571, 24028670, 19244539, 19244527, 1222108, 1222098, 1222094, 24028618, 1222090, 19834206, 17540417, 15181067, 10396893, 19834000, 19833956, 19833926, 23438361, 10396645, 19833786, 19833782, 19833768, 10003300, 10396237, 17015309, 16949750, 20554155, 23175577, 16097675, 23437703, 22192508, 23241054, 10395963, 1220790, 1220772, 166174784, 11313177, 10395645, 10330109, 20553657, 2662309, 2662289, 11313003, 7970663, 3972930, 20553523, 11312883, 19832520, 18849432, 19832420, 3972684, 4103751, 10395201, 19832374, 11312685, 4103649, 19832206, 19832204, 166174088, 14785859, 1150000443, 1150000441, 19832100, 1678607, 10394839, 11312159, 23239708, 19831836, 19831786, 10329053, 24288178, 24288162, 20749145, 24287930, 24287868, 10001010, 15702637, 24287830, 20355620, 11311633, 10328539, 11311577, 24287662, 10328483, 10328475, 10328453, 10328437, 10328427, 10328411, 24287572, 24287548, 24287496, 24287468, 11311255, 11311247, 24287326, 11311171, 11311167, 4560930, 11311135, 11311053, 11311049, 11311015, 1611680, 19830654, 11310973, 10000114, 11310773, 10720900, 10720870, 10720856, 10720850, 4036168, 19830344, 19830290, 23893491, 19830226, 19895747, 19895715, 24155548, 12752271, 19830156, 19830140, 4887885, 14521635, 15111455, 10327265, 10327255, 20681923, 10327229, 4559956, 10785842, 19829680, 19829678, 21402507, 17339243, 9999208, 9999172, 4035378, 9999042, 23499451, 21861024, 11440785, 17338937, 10719764, 23499231, 19829184, 10719670, 6263212, 6263182, 10719622, 10719598, 10719588, 5542230, 10719538, 11178255, 10719454, 12554446, 19828932, 19828918, 16224377, 3379321, 11112557, 6721637, 18911319, 11112533, 12685393, 18911309, 6262668, 21860192, 22187833, 11177763, 11112159, 11177681, 6721221, 22187711, 11177653, 10718900, 6131373, 948080258, 11177593, 11177559, 6721037, 21859834, 1412597, 22187475, 3575230, 23104858, 23104836, 22383887, 23104752, 23104748, 22514922, 10718416, 11111533, 10718244, 9997302, 7900047, 10718058, 10718050, 10718048, 10718014, 10717988, 17599227, 17599217, 23104012, 2787839, 14584315, 14584231, 14584225, 2787725, 4032830, 14518541, 17664267, 4032772, 11438281, 14518455, 20351101, 3573878, 20351079, 11700294, 20351045, 6129721, 936020015, 10782766, 10782706, 11438065, 6719467, 11438031, 6719419, 11175799, 10520410, 10520406, 10520384, 21530408, 14518029, 10520328, 10389248, 10520318, 24282854, 23430870, 17335981, 23103126, 17335907, 5867099, 1214037, 1214029, 1214013, 10520070, 1213947, 24544679, 20546958, 20546932, 11175207, 11175175, 4621524, 22381745, 17663125, 5866575, 10716132, 23233459, 3572613, 10715906, 7897853, 16745193, 23954037, 10715754, 10715738, 14254679, 23953949, 23953947, 1212943, 24543741, 18514384, 10715592, 10715590, 1212773, 10715480, 23101670, 23101622, 24019117, 20545680, 3178536, 15106053, 23101394, 14253949, 20217628, 11763414, 23101086, 1736350, 294509, 23100908, 23952792, 10780046, 23952776, 10780040, 21527932, 10452348, 23100642, 23887038, 10320932, 17595303, 24542027, 10451774, 23886640, 11303718, 23886594, 23231145, 23231077, 17594813, 6781247, 11565356, 11565340, 11565326, 1734906, 10713316, 14252257, 2390104, 1407047, 6322161, 6322157, 13072338, 5863363, 20215696, 13137643, 17593953, 13071946, 19887657, 19625466, 19887607, 6321621, 11498936, 2651566, 24081835, 22770922, 21066956, 21066948, 22770864, 19887261, 5862551, 19625084, 10777624, 18772984, 23950285, 17593291, 12088263, 12088255, 17593261, 21328785, 21328783, 23950177, 6320977, 23950105, 23950079, 23950021, 12087951, 20541994, 19886567, 20541922, 19886513, 10777006, 10776890, 12087603, 12087595, 20541736, 6123761, 19886319, 11432173, 11432167, 20541662, 10318006, 5927064, 12087417, 12087413, 23228442, 11431951, 12087297, 12087293, 12087279, 12087271, 12087223, 20803433, 12087139, 12087109, 12087045, 12086999, 12086961, 12086947, 22769312, 12086925, 11759194, 24211031, 22769178, 5860871, 5860861, 19885563, 19885549, 22769122, 17657311, 24538568, 22769084, 17657259, 17657249, 17657227, 17657205, 22768922, 20540606, 5205156, 9989269, 6646900, 6777941, 24210509, 18967610, 18967576, 11758608, 5204986, 17591268, 17591264, 13069258, 5204910, 17591156, 13069164, 5204812, 21850951, 13069118, 5204758, 2976470, 12085945, 12085925, 6711917, 25193054, 4287059, 24209981, 24406485, 20867490, 24406323, 24406303, 1337578, 9988311, 12085461, 92296, 92284, 17590290, 24274951, 17590258, 92108, 92088, 12085167, 92066, 11364249, 17590168, 11364247, 17590152, 11364217, 17590122, 17590118, 17590108, 20866896, 17590096, 16934598, 17393334, 16213598, 20866648, 17393182, 20866580, 17393114, 22635990, 22635982, 22635974, 24274357, 24274315, 16213364, 5203252, 16999704, 16999696, 11363573, 20604076, 20604062, 14181518, 17785981, 20276295, 1663907, 1663903, 11363227, 20800405, 7299964, 20341543, 1401516, 1401494, 22635148, 11362897, 22634964, 11362719, 17785181, 21061980, 17785135, 10182953, 11362539, 17785065, 17785023, 17785009, 17785007, 22634636, 11362439, 11362383, 24535102, 11362365, 10182691, 14639091, 24534980, 24534964, 24534914, 17784671, 23027542, 89880, 16932618, 24534670, 8412775, 20536924, 1990228, 89600, 23027180, 1203612, 16604487, 8412487, 20536596, 24534268, 20536568, 20536532, 10181831, 20536516, 20536512, 20536498, 1203364, 8281251, 16604301, 89220, 9985085, 16931822, 20536208, 17586938, 17586906, 11688588, 24271478, 9984489, 20666846, 16931292, 11360475, 13654178, 23156892, 6772889, 24270968, 24270920, 9328678, 14702606, 24074245, 22632372, 24074017, 22632068, 11359869, 11359859, 5330541, 20534882, 5330515, 11359817, 5330445, 24073679, 14570934, 23877033, 8410533, 5002653, 6641004, 20665688, 20665680, 4937035, 16078064, 11359445, 8410309, 8410307, 20665516, 20665486, 19616876, 20206686, 17716316, 16077902, 13784114, 20206622, 20206602, 24073209, 17716208, 20206560, 1201116, 20206546, 11228090, 11359119, 11359105, 14439236, 20599612, 11359031, 1987376, 20599598, 1200928, 20599546, 20665038, 11358923, 12669574, 5329541, 1200762, 6181470, 1987158, 5329487, 20664910, 17715744, 20664862, 5329431, 22761992, 11227622, 5329369, 14045653, 16863677, 14045611, 20664740, 22630788, 9326970, 17715390, 17715244, 22826894, 13782914, 1986382, 1986360, 1986356, 1986346, 12668708, 1986328, 20467458, 20205242, 14175921, 20205180, 14175847, 931090014, 16928318, 20467232, 20467218, 16928260, 23023106, 16928258, 22826492, 22826480, 22826428, 24399274, 478555, 478541, 3231045, 16928044, 16928030, 16928012, 16928008, 478369, 478363, 478359, 478333, 20597872, 16927822, 11160643, 21384110, 22629178, 22825784, 1985292, 19876608, 14961393, 1985262, 24595173, 23022274, 1985216, 23480975, 215671, 21383632, 1984960, 13781414, 21383450, 1984766, 22628574, 19810523, 4868307, 19810511, 22628530, 22628488, 19810423, 20531303, 22628450, 19810401, 20465748, 18303038, 21907503, 22628390, 20531207, 22628358, 83972, 1394679, 20531189, 20531173, 22628280, 20596632, 20531075, 20531047, 14042893, 19875584, 19810045, 17843960, 20530919, 18302666, 1984162, 20530831, 1721983, 20792954, 10176091, 1197592, 83480, 22824452, 6178230, 19809717, 1197430, 1197400, 19875136, 22824246, 19809589, 19875116, 1983774, 1197234, 18302102, 2638994, 18302090, 19874948, 17450105, 24003677, 11682859, 9454571, 18301928, 1197032, 19874776, 18301872, 19874734, 18301856, 18301784, 24462150, 1983286, 19874556, 24986362, 18301664, 22561416, 3359340, 19874370, 7946806, 19808789, 18301452, 19874300, 19874296, 21905881, 22823384, 3359184, 22823358, 22823344, 2245035, 22823304, 14827891, 21250406, 14762333, 19874102, 20791599, 2179341, 21250302, 9453821, 11878624, 2179269, 5849248, 11157589, 21250100, 19808305, 19808271, 19808159, 22822808, 22822796, 22822748, 12664654, 12664652, 23871303, 22822706, 22822492, 22822476, 22822470, 22822446, 19807789, 11157027, 2899473, 2178567, 10960108, 4340936, 10959952, 10959934, 6241337, 19872794, 8403979, 18693087, 24001501, 8403933, 24001441, 1588093, 21248876, 24001313, 19807009, 24001311, 24001309, 4930212, 7879320, 22821514, 11877000, 22821492, 22821488, 6765150, 8403537, 22821382, 8206819, 801243, 8206795, 3160505, 13121864, 22624582, 22821166, 22821162, 19806373, 4470829, 21247986, 19871686, 19806097, 22624144, 22624118, 19806049, 20854431, 20854385, 3159665, 12662344, 10696177, 5846482, 9450945, 22820268, 4142500, 5846384, 19805489, 22820140, 20526337, 8402171, 22820080, 20657292, 11285640, 11285638, 20526173, 20657228, 20657208, 10695651, 19804945, 22819574, 19804869, 11547300, 7746107, 21901825, 20722130, 11284910, 24129937, 4141456, 20853103, 11284792, 14495988, 23277746, 14495894, 20852879, 10694755, 20852827, 5845056, 10235923, 20852725, 19804081, 7221154, 20524939, 20524889, 18886480, 21376806, 21376800, 20655874, 20524789, 20524759, 10170030, 10170028, 11677337, 14495356, 11677281, 11677227, 11677211, 11677209, 21245419, 11283938, 11283932, 20655506, 4468075, 22818140, 4468059, 4468037, 23997748, 15871281, 19803439, 22818094, 15871257, 19803403, 19803399, 19803397, 23997696, 23997670, 19803361, 19803349, 20393165, 23997634, 17706159, 17706153, 945040542, 5844098, 17706109, 23866478, 20851817, 17443946, 23866458, 21245013, 18295880, 22555662, 20392971, 23866342, 17443814, 20851649, 20392863, 17443734, 9448335, 8924035, 17443700, 23866226, 20392817, 17705829, 8203105, 17705825, 17705823, 11676409, 10234609, 5253823, 11152043, 23866002, 23865988, 23210530, 19802657, 19802651, 11676179, 11676103, 8202619, 23865722, 16918842, 11675941, 8202525, 16918788, 1386706, 17705167, 1190013, 1189991, 23865444, 20392017, 23865412, 1189933, 8202255, 8202237, 22226934, 20850675, 11478998, 16918358, 24061761, 10168128, 10692407, 22882073, 23209726, 6170364, 6170354, 16918238, 22882009, 26814119, 16918178, 16918036, 21374482, 10167814, 10167812, 10167804, 19080677, 20653460, 23209348, 21374204, 21374202, 14886113, 1188991, 9446517, 20849691, 8922095, 20849625, 8922069, 23208906, 21832459, 2827008, 9446133, 8200931, 10232491, 17703555, 26747502, 26747474, 20849173, 1188347, 1188345, 21438933, 5251539, 1188291, 9445761, 6168832, 6168828, 6168824, 20848839, 18292932, 1188033, 10231965, 9445525, 5251167, 14426184, 1187879, 10231709, 1187707, 10231665, 18292552, 9445175, 1187633, 7151398, 1187591, 11476728, 1515223, 2236083, 1187467, 21306991, 1187423, 21437987, 1187337, 13311451, 11083215, 5250479, 18292138, 14491032, 9903509, 18292116, 18292104, 1383758, 2235723, 24059187, 20585778, 1187091, 1187083, 6561019, 18291938, 1187007, 11279494, 18291776, 14490612, 943070173, 943070156, 8919967, 16128891, 21437219, 9509648, 9444107, 11082499, 20585196, 24386189, 17635974, 21437057, 21240340, 21240318, 18291192, 21240280, 18291158, 21240250, 18291090, 9509158, 9509124, 19011821, 20715710, 18290876, 24058019, 14751896, 9443475, 8460436, 14751882, 9902207, 948050044, 23664761, 20715614, 23664707, 23861301, 8460308, 8460296, 20518917, 23861217, 21436293, 21436273, 21436259, 8459936, 10163852, 1185327, 17307152, 21435881, 21435771, 14685538, 14685516, 22811961, 14685468, 14685378, 10753211, 18355349, 10753107, 9442347, 9442225, 17372052, 9507722, 20517665, 10752793, 23991063, 1184517, 10425032, 23990965, 23990893, 21303915, 21434949, 14488118, 21434933, 14750252, 6689283, 21303807, 14750196, 16126367, 23990647, 21762422, 14750002, 21303583, 24383771, 23990545, 23990539, 11473150, 23990523, 21762270, 14749902, 23990475, 2298052, 266419, 23990437, 21434529, 23990423, 21762176, 23990371, 23990327, 23990313, 13635621, 23990251, 16125927, 23990173, 4001688, 23990135, 266101, 23990119, 23990115, 266083, 266039, 266009, 11210518, 17370894, 11341569, 20516605, 11210492, 23989989, 11341539, 23989985, 265951, 11341533, 23989975, 20516563, 265935, 265909, 17764001, 11210362, 14945906, 11210306, 23989819, 17370682, 14945776, 11210224, 11210216, 17173960, 17173920, 17173804, 17173802, 16977170, 17370378, 11209978, 17173700, 11340981, 23989397, 11340871, 11340849, 23989293, 11340827, 11209754, 17370100, 11340785, 23989191, 13503428, 22678436, 17763227, 22678414, 17763205, 19008361, 17763147, 11078389, 22678256, 11078295, 19073673, 19073659, 68194, 11340369, 17762829, 27724291, 17762757, 17762453, 21760148, 11602053, 20318298, 20318294, 11602005, 11601991, 17762371, 17762359, 17762311, 19072997, 20318174, 11601885, 17762265, 11601861, 11601849, 19859378, 11601837, 19072941, 20318122, 11601807, 10749821, 10749805, 22939350, 21759664, 19859076, 17761905, 17761887, 21759562, 11339239, 13501908, 13501902, 17761741, 17761715, 10814866, 11339149, 14484850, 7144796, 24839495, 14484800, 14746908, 11338993, 11535547, 9503926, 11207858, 10683562, 7865506, 7734415, 10683524, 22676576, 23135321, 14156760, 14746552, 17171366, 9372564, 14746394, 23134995, 8913659, 23134895, 14156402, 10093116, 940180013, 8192501, 16974322, 14483942, 23921086, 10682718, 18350401, 11338033, 23920868, 23920838, 16056442, 8192111, 7929813, 9436939, 17432240, 18349627, 9305572, 7732701, 20381117, 20381115, 12058006, 12058002, 12057980, 9305414, 9436483, 9305360, 7732493, 12057850, 17693945, 2489414, 13499382, 12778377, 3341170, 3341128, 11467574, 11467564, 23722671, 7076514, 8190581, 10680910, 17889686, 10484105, 3340616, 3340610, 3340602, 3340598, 2685216, 17692949, 16185602, 23066855, 23066809, 3340462, 23066795, 23066785, 23066783, 8190101, 8190089, 938080243, 17037279, 3340184, 3340122, 3340108, 1373852, 8386143, 16971296, 16971274, 9500074, 9500056, 9500024, 9500008, 9499974, 9499930, 8189187, 8189185, 8189183, 9499882, 9499848, 8189083, 9499744, 9499726, 9499664, 17888102, 8188707, 8188687, 8188683, 25228000, 25227994, 8188631, 25227990, 1897168, 9499334, 1897148, 2683528, 9499112, 2683348, 9302483, 2683328, 2683302, 2683298, 16970140, 9499010, 2683182, 2683162, 8384639, 2683004, 17887314, 2682942, 2682804, 17690533, 2682786, 9498498, 24440664, 8187681, 9629405, 8384161, 8384159, 8384149, 9629303, 8384113, 9629271, 9629261, 9629253, 9629211, 9629177, 9629171, 1371618, 17296828, 9891212, 2682210, 2682200, 2682196, 17165578, 10480417, 14412443, 8317535, 11463244, 24046147, 24045801, 1698016, 9496752, 944040015, 944040014, 2680862, 22996991, 7071686, 1566660, 22603713, 7071680, 7071678, 22996923, 9496494, 20113248, 14411563, 14411561, 9496342, 9496314, 9496222, 9496196, 2680450, 6743670, 2680404, 9496054, 9496046, 9496044, 16770460, 17687807, 16770174, 16770172, 16770122, 16770118, 2483204, 16770042, 16770034, 16770014, 8315775, 8315765, 8315689, 8315687, 8315683, 9626371, 22602485, 9626329, 8315557, 8315541, 9495162, 9495158, 23978366, 9888022, 10477667, 10477205, 4906637, 10477185, 10477159, 2678298, 10477069, 10477055, 10477053, 10477015, 10477001, 10476997, 10476985, 10476973, 6151469, 1563936, 1432855, 1563914, 948100355, 23059710, 23059698, 23059696, 6741182, 1432765, 23846075, 1432757, 17161360, 18275247, 14277540, 25156363, 25156359, 22534696, 16767388, 11393268, 448730, 16767180, 2676896, 8378249, 8378235, 11458056, 2676068, 2676064, 18076994, 35378492, 1692972, 18076954, 11982083, 11850997, 22598597, 1692610, 1692580, 18076528, 18076438, 18076398, 18076392, 2675408, 17158852, 23319198, 8835742, 21746217, 2544133, 25088437, 8376673, 17682766, 8376653, 17682762, 8376637, 8376631, 8376627, 21745949, 2543895, 11981053, 2543867, 17682678, 19779815, 7917745, 17682518, 17682346, 8375981, 11390614, 11390576, 10407344, 3853725, 2804937, 2804931, 2804711, 2804639, 2804637, 2673496, 2804487, 11389511, 3262979, 24037788, 14600567, 14600563, 14600561, 14600559, 2803767, 3852187, 18073472, 11323172, 11323160, 23774899, 4900431, 23774729, 20301203, 18072904, 4769018, 4769016, 4769006, 24036578, 4768974, 4768958, 4768950, 4768918, 18072596, 3195900, 18072544, 24036184, 24036178, 18072314, 18072242, 23839318, 5882397, 22069764, 11321858, 7913960, 7913956, 7913914, 7913912, 3326364, 5882197, 5882189, 5882181, 5882179, 5882177, 5882173, 5882161, 5882155, 24559885, 16760920, 5881825, 6078409, 17809276, 17809272, 19906316, 19906314, 5029567, 24559145, 24296954, 9485694, 16039274, 16039138, 16759996, 24493216, 17153002, 3324878, 4766652, 4766650, 4766546, 16759602, 4766506, 16759548, 16759536, 4766444, 15448630, 16759338, 3389931, 3193068, 948091065, 18331654, 24885173, 18331514, 1357564, 2078437, 20297439, 15447582, 15447580, 15447578, 5289491, 20362625, 19772775, 22656333, 5879055, 9483502, 10466517, 5878909, 5878901, 5878885, 12694550, 10466323, 5878751, 5878749, 5878747, 5878743, 5878741, 5878739, 5878723, 5878721, 12694456, 10466205, 5878677, 10466179, 10466177, 5878655, 5878649, 10466169, 10466121, 5878595, 5878573, 5878571, 10466063, 10466035, 10466023, 10466021, 10466001, 6730429, 242204, 22655477, 17150450, 25211355, 25211343, 17150354, 22393109, 6729923, 24359003, 24359001, 2666353, 15183569, 15183567, 15183527, 18329038, 19836220, 19836210, 5876973, 5876967, 5876955, 19836062, 22916088, 19835614, 1354312, 15968803, 1354238, 2664645, 23701689, 2664621, 2664593, 2664555, 2664515, 11446253, 20359118, 1222572, 23701415, 23701411, 2664309, 22390611, 21276484, 22783728, 24028768, 24028764, 24028640, 24028634, 2663717, 19834048, 10003500, 10003488, 24027768, 20226589, 23241096, 16949142, 10854271, 2662155, 10854139, 18521839, 10854125, 2662119, 18521831, 10854091, 10854089, 10854061, 10853995, 10395245, 2661923, 2661909, 1678863, 3251405, 10329251, 22781082, 24288218, 24288214, 24288208, 24288184, 24288164, 24288134, 16816981, 24287894, 1743034, 11311107, 10327587, 24155554, 21271934, 20222893, 23499611, 23499597, 23106346, 23499251, 9409002, 6262942, 6262900, 22515758, 23957392, 6262660, 11177859, 1412969, 6721235, 6131379, 6131377, 6131349, 6130851, 6130849, 17140874, 17140872, 14584825, 942050097, 11438323, 11438285, 11438279, 11438275, 11438257, 5867681, 5867639, 10389612, 6719555, 10520590, 6719459, 16746445, 16746433, 7702458, 6719415, 6719413, 23234469, 10520374, 11503398, 10520340, 10520216, 6129295, 10520204, 6129291, 6129283, 3901053, 6129247, 6129231, 6129225, 6129147, 167806458, 6129139, 6129137, 6129111, 6129025, 5866865, 5866837, 5866785, 944080037, 944080036, 5866635, 6325377, 17663085, 23954433, 1737704, 16745395, 1672029, 16745277, 16745255, 20153109, 20153075, 16745199, 20153073, 16745187, 16745185, 16745175, 16745135, 3179082, 23953989, 23953967, 23953965, 23953955, 24543745, 24543743, 1343922, 10715566, 1671597, 3178876, 20742138, 1408939, 2785081, 24018715, 23952752, 166755566, 11566318, 17136858, 11566070, 1407911, 17594919, 17594907, 17594787, 17594769, 17594757, 17594751, 1407337, 17594717, 19888453, 5863725, 11565344, 1734750, 6322215, 6322213, 5863393, 23361233, 11499140, 11499138, 11499136, 19887641, 19887639, 3175744, 4879651, 19887371, 9991271, 6714463, 4289595, 10712040, 11432873, 17003328, 12087965, 3240533, 20803657, 6713405, 3239905, 6778631, 24538680, 5860839, 5860781, 5860753, 948100613, 22899788, 5860425, 5860419, 5860411, 5860393, 13069330, 17591310, 5204960, 17591262, 17591258, 17591250, 5204942, 17591246, 17591236, 5204908, 5204872, 5204830, 5204704, 5204686, 1337982, 6711903, 24210011, 24209967, 11561424, 11561412, 12151156, 11561316, 1337666, 1337520, 16935038, 16935036, 17590336, 92124, 92082, 16934620, 16934614, 91726, 17589820, 22635650, 18375528, 16933516, 16867821, 24535464, 23028110, 23028088, 936010044, 16933118, 16933056, 10051650, 24535104, 163864576, 24535030, 1204144, 24534920, 25386875, 6315885, 89906, 24534830, 16932652, 16932626, 24534714, 24534704, 89658, 16932358, 89570, 89556, 16932302, 89464, 8412523, 16932154, 89316, 9985221, 9985219, 9985213, 89270, 89210, 8281201, 89190, 89176, 8281147, 8281129, 16931868, 16931846, 16931818, 8280803, 16931374, 22633002, 16931214, 22632778, 6772989, 24664249, 24664247, 24271026, 14768266, 22763430, 6772593, 10245973, 24073989, 87628, 5330485, 5330473, 6772233, 5330435, 8410595, 8410591, 8410331, 8410305, 8410287, 8410215, 8410159, 3560472, 22762510, 5329933, 22762504, 8410111, 20206564, 24073133, 1987486, 22762326, 1987404, 1987364, 1987358, 1200902, 22762218, 6181460, 5329421, 5329361, 5329329, 4608414, 22630804, 22630802, 22630776, 16863605, 33771791, 4215004, 24400066, 8409117, 8409055, 1986498, 9326320, 1199840, 1199838, 8408603, 8408577, 478661, 478643, 478595, 16928072, 11160891, 16928050, 8932628, 16928014, 478467, 478461, 478405, 478367, 478357, 478343, 16927850, 16927848, 16927836, 16927786, 16927766, 3230645, 1985350, 1985332, 1985306, 1985302, 1985296, 1985266, 1985242, 1985238, 1985222, 1985190, 1985174, 1985164, 1985158, 1985100, 1985088, 1985046, 1985040, 1985024, 1984996, 19810779, 1984978, 1984950, 1984926, 1984900, 1984758, 1984706, 16861377, 1984642, 1984630, 5195870, 933120058, 1394723, 83974, 83968, 7292856, 7292852, 1984356, 3360590, 3360588, 19875632, 83606, 1984146, 2180751, 2180727, 83560, 2180629, 24004077, 9454935, 1721657, 19809555, 4605154, 19875004, 6177954, 6177950, 2180235, 21906569, 2180149, 2180147, 3359776, 1721283, 4604838, 21906305, 3359534, 3359492, 1983144, 2179701, 3359316, 3359284, 21905903, 1982950, 7291250, 22102361, 2244891, 2179353, 20529413, 2179323, 2179317, 19808485, 2244829, 2179275, 2179251, 2179245, 2179235, 2179223, 2179175, 2179169, 2179137, 2179121, 2179111, 1523651, 22626154, 22626100, 2178647, 22822448, 14826999, 14826985, 14826977, 14826971, 20331954, 9452935, 10960058, 10960004, 20331614, 6241365, 20331586, 10959928, 10959688, 8403559, 6765046, 801229, 801217, 801215, 3357024, 800857, 800727, 800725, 166737764, 166737763, 944060065, 8074898, 10499663, 23541178, 10040680, 10040678, 8074578, 22819896, 6763422, 21901983, 5845564, 8401459, 21901869, 21901847, 21901839, 8401419, 1978764, 1978762, 1978704, 20590712, 23015230, 5844738, 11677391, 4599299, 20852201, 5844452, 20852145, 23997802, 23997800, 23997764, 23997714, 17706197, 21245047, 23866256, 9448273, 23866180, 23866030, 23866010, 20261505, 23865978, 10234349, 11676137, 23865798, 19081491, 16918648, 22227036, 22227004, 206907, 23865396, 16918392, 16918350, 16918316, 9316083, 6170350, 6170348, 6170346, 26814135, 16918162, 5842532, 16918072, 16918002, 16917974, 1385796, 9446677, 16852178, 9446565, 16852134, 16852118, 9446513, 9446423, 3154793, 17703681, 3154655, 26747494, 16851536, 941040195, 26747450, 15999542, 1188363, 16851450, 7151998, 18293060, 1188133, 6168860, 6168852, 23142591, 24453257, 18292858, 1187933, 10231725, 23142299, 16916370, 16916364, 16916362, 9445257, 9445131, 16850684, 9445105, 1515237, 1515231, 1515217, 17899183, 9445039, 18292380, 18292356, 2236017, 17374704, 18292194, 18292152, 18291780, 11082661, 17374058, 9509692, 9313083, 9509668, 9509658, 18291472, 9509646, 11082461, 20585096, 9443935, 9443911, 11082197, 9509292, 9509276, 24058195, 9509160, 9509156, 9509140, 9509138, 9509118, 18290928, 24058089, 18290914, 12327115, 9902267, 18290868, 16849022, 24778867, 24778865, 202861, 16848968, 8460316, 14751714, 11802477, 11802475, 9901791, 9901773, 22681238, 14685758, 9508298, 8459716, 22812097, 18290112, 22812071, 14685570, 16979258, 9442443, 9901143, 17896445, 17896441, 23991285, 9507578, 23990665, 23990633, 23990557, 2298064, 2298060, 23990333, 9441335, 23990325, 24186852, 23990233, 2297774, 2297770, 23990159, 23990141, 12586873, 23990029, 23990003, 9375446, 23989949, 23989905, 8457776, 23989807, 23399979, 1314089, 24186102, 23989459, 23399533, 23268322, 8457024, 11078369, 11078357, 68292, 27724429, 7932537, 27724393, 8456582, 8456566, 7866734, 8456540, 17172812, 24185122, 11602055, 11602049, 11602007, 11601985, 11601981, 11601973, 11601955, 20318230, 7866376, 20318168, 1705874, 9438927, 8193669, 20579969, 22808049, 7865790, 24708526, 18351513, 7865742, 1574262, 14746916, 14746914, 14746912, 14746910, 14746906, 14746894, 14746892, 14746886, 14746880, 14746870, 7865536, 9503916, 7734431, 12584027, 14746532, 23135091, 8192849, 23921176, 16056760, 14352318, 12058548, 12058542, 12058540, 12058536, 12058534, 12058520, 12058514, 12058498, 7733103, 7733101, 13500192, 7732957, 7732833, 7732815, 12058186, 9436713, 7732745, 19856892, 12058106, 12058092, 9436627, 17890732, 17890724, 12058008, 12057994, 17694057, 12057960, 9436477, 2686254, 2686246, 7732483, 9436397, 17693921, 19856606, 7732437, 17693903, 8191181, 8191167, 7732399, 9436331, 7732389, 7732377, 7732373, 7732369, 7732367, 13499426, 13499386, 7076848, 941030377, 17562596, 20380627, 3341254, 3341250, 3341246, 3341242, 3341240, 3341140, 2685780, 8190797, 8190731, 3341046, 3341028, 8190683, 17890006, 17889984, 8190625, 2685526, 8190549, 10680912, 2685518, 2685514, 8190527, 2685486, 8190507, 19855912, 2685452, 10484165, 8190399, 8190363, 8190359, 10484103, 1702276, 8190291, 8190279, 8190259, 8190213, 17692925, 8190195, 17692847, 17692829, 17692813, 3340404, 17430530, 12056532, 3340174, 3340172, 3340168, 3340164, 8189817, 8189807, 3340128, 3340120, 3340118, 3340102, 8189761, 1439509, 3340044, 10024690, 13891288, 1636001, 8386139, 9500164, 4912616, 5240120, 8385797, 8189151, 8189107, 8189095, 8189081, 9499802, 943060111, 8189065, 8385671, 8189027, 8189025, 8189021, 8385607, 8385591, 9499690, 9499686, 12055588, 10482701, 12055564, 9499650, 14414849, 10482687, 9499644, 8188923, 10482677, 10482665, 10482663, 10482619, 8188857, 8188847, 17888080, 12055316, 8188679, 9499382, 8188655, 9499350, 9499324, 9499314, 2683530, 2683526, 9499264, 2683510, 11334261, 2683502, 2683490, 9499222, 2683448, 12055054, 19853838, 2683366, 9302487, 19853782, 9499092, 2683346, 2683326, 2683322, 16970168, 2683318, 2683310, 2683308, 2683300, 10482083, 2683290, 2683282, 2683276, 9499020, 940110218, 940110217, 2683258, 10482041, 2683252, 10482033, 2683226, 2683176, 17297702, 9498916, 17297688, 2683158, 9498902, 2683146, 9498888, 2683130, 2683118, 2683108, 2683102, 2683094, 1700052, 2683084, 9498810, 2683050, 2683044, 2683036, 2683034, 2683020, 8384647, 9498758, 2682998, 17887344, 2682982, 2682970, 2682966, 17887318, 2682958, 9629755, 9629753, 2682926, 2682922, 2682920, 2682918, 2682908, 9498650, 9629717, 2682900, 9629715, 2682894, 2682886, 2682884, 2682882, 9629691, 17100763, 2682840, 2682838, 2682832, 2682820, 2682812, 2682802, 2682790, 2682788, 2486178, 2486176, 2682778, 2682770, 2682764, 9498486, 2682710, 9498454, 9498432, 11398962, 8187677, 9629469, 8187667, 9629417, 9629411, 17690337, 9629357, 17297068, 9629337, 9629325, 9629315, 9629307, 10677878, 9629301, 2485870, 9629291, 10677862, 9629285, 9629283, 9629281, 10481241, 9629273, 24964694, 8384041, 9629185, 9629179, 9629173, 17296830, 2682206, 2682204, 2682202, 11332847, 1174468, 1174450, 1174444, 10480441, 20310669, 1174094, 14412355, 14412345, 14412333, 14412331, 8317367, 24045821, 8317111, 10479779, 10479777, 8317045, 17164340, 2680876, 1566648, 947120049, 9496488, 9496462, 9496460, 10479427, 10479415, 10479411, 10479369, 9496316, 9496306, 17688279, 1566402, 1566398, 10479269, 9496230, 10479251, 2680458, 15853185, 2680440, 938010228, 11396694, 2680406, 2680402, 2680398, 16770624, 2680368, 2680364, 9496102, 11331043, 11986401, 17688031, 17688017, 2680212, 21423488, 17687905, 16770396, 17687899, 17687893, 10478927, 16770380, 16770376, 15852781, 16770194, 22996111, 16770180, 16770176, 16770138, 16770126, 16770108, 16770078, 16770064, 16770062, 16770060, 17687545, 17687543, 17687541, 8315783, 17687427, 2679670, 17687385, 17687383, 11395872, 9626369, 9626367, 9626365, 22602487, 9626353, 9626331, 2679482, 22602385, 8315535, 9626251, 9888392, 11395718, 9888386, 9495148, 9495144, 9495134, 11395540, 23978420, 10674544, 10674522, 11984715, 6676158, 945020595, 10477233, 1564304, 945020556, 10477197, 10477195, 10477193, 10477191, 10477187, 10477183, 10477173, 10477167, 10477165, 10477161, 10477157, 10477155, 10477147, 10477145, 10477143, 5693012, 10477141, 10477125, 10477123, 10477119, 10477109, 10477107, 10477103, 10477101, 10477099, 10477097, 10477095, 10477073, 10477061, 10477059, 10477057, 10477049, 10477047, 10477043, 17161708, 16768484, 10477025, 10477021, 10477019, 10477013, 10477005, 10477003, 10476993, 10476991, 10476989, 10476983, 10476981, 10476979, 10476977, 10476975, 10476971, 10476969, 10476967, 10476961, 10476959, 10476957, 1564034, 11984227, 1563918, 1563890, 1563888, 1563884, 1563882, 1563880, 1563876, 1432797, 11394264, 1432761, 17161374, 23846011, 23846005, 23846003, 23845983, 937090113, 937090100, 2677764, 2677750, 2677724, 2677712, 17619892, 1170350, 14277542, 17619878, 17619860, 17619858, 1170304, 16767830, 940170018, 23845641, 2677500, 23845619, 23845595, 3529407, 2677320, 2677316, 2677306, 2677304, 23910927, 2677190, 11983281, 11983255, 5953897, 23910749, 2677062, 23910693, 2677012, 23910659, 23910645, 11393226, 2676934, 2676906, 2676904, 2676894, 2676892, 11851899, 2676856, 8378479, 7264329, 2676788, 2676776, 2676732, 2676728, 2676690, 16766918, 972734, 2676636, 17684366, 8378251, 2545547, 448396, 8378233, 8378223, 2545515, 2545513, 8378211, 8378207, 22599519, 8378205, 8378203, 22599511, 8378197, 8378193, 8378185, 940300103, 2676548, 8378173, 8378165, 2676502, 16963283, 1562226, 1562186, 2676184, 2676174, 1693132, 2676170, 2676162, 11851163, 2676092, 11851077, 11392306, 1692976, 18076960, 15193339, 11850999, 2544871, 2544869, 2544863, 2544815, 2544737, 2544721, 2544671, 11981811, 941020117, 941020116, 2544515, 18076526, 18076524, 18076502, 19059514, 19059468, 19059466, 18076390, 18076386, 2675418, 18076376, 2675410, 2675406, 17158860, 18076344, 8835746, 7262835, 7262833, 2544227, 3527263, 11850319, 14930481, 8376871, 8376863, 8376859, 2544111, 16765354, 8376709, 8376687, 11850095, 3527015, 8376675, 8376661, 2543879, 19779827, 1691846, 16765050, 19058680, 17682350, 17682270, 17682200, 2805487, 9359072, 10407512, 20368894, 2805043, 2805007, 2804953, 2804927, 937020605, 937020587, 10406992, 10603564, 2804763, 2804743, 2804729, 2804713, 2804699, 2804655, 2804643, 2804627, 2804577, 10734430, 2804551, 10406712, 2804533, 2804517, 948030233, 2804411, 2804389, 2804329, 2804299, 1558968, 1558960, 2804039, 14600493, 1558826, 1558814, 1558812, 1558810, 1558792, 9619679, 2803927, 2803913, 2803901, 2803851, 11323478, 11323474, 11323470, 18073430, 18073428, 945080120, 9357014, 9357006, 15713819, 15713815, 9356778, 940230070, 22070704, 940230045, 18072988, 18072984, 18072982, 18072980, 11388287, 18072948, 11388231, 11322694, 18072900, 18072826, 11191541, 4768980, 10405010, 10405008, 3851383, 23839824, 23839822, 14599227, 18072626, 18072606, 18072598, 14599187, 20300809, 20300807, 18072566, 20366305, 18072532, 14599123, 14599115, 14599109, 11125677, 23839556, 20366141, 5489453, 20366125, 18072350, 18072344, 23839486, 11125487, 18072276, 18072270, 14467779, 11125387, 11125383, 20365897, 8962618, 11321906, 23839268, 11321874, 22594061, 11321868, 11321864, 11321852, 11321850, 22069746, 23839198, 11715035, 24494536, 23839082, 23839038, 23839036, 5882171, 23839034, 11976591, 20365199, 20365195, 20365181, 20365175, 20365163, 20365161, 20365155, 20365143, 20365133, 20365129, 5029565, 17743431, 9485712, 16039286, 16039232, 16039142, 10402973, 10402963, 2538569, 2538567, 17153048, 4766688, 17152972, 20429750, 9485232, 9485230, 9485228, 9485226, 9485218, 9485202, 11123573, 4766542, 4766492, 10402579, 20429572, 14596809, 9484972, 4766362, 10402355, 10402353, 10402347, 10402345, 10402341, 10402339, 10402337, 10402335, 10402189, 10402181, 10402177, 20298091, 10402139, 10402137, 10402121, 10402101, 23640352, 10402055, 10402047, 10402041, 10402025, 10402021, 10402007, 22657169, 948090999, 21084260, 21411887, 21411865, 948090883, 23640064, 1357802, 1357764, 948090788, 18331534, 1357698, 18331496, 18331492, 18331480, 18331478, 948090655, 20297493, 20297441, 945010348, 22656571, 20362811, 20362809, 20362797, 20362795, 20362793, 20362773, 20362771, 20362765, 20362763, 20362761, 20362759, 20362757, 20362755, 20362753, 20362749, 20362745, 20362743, 20362735, 20362723, 20362721, 20362719, 20362717, 20362715, 20362707, 20362705, 20362703, 20362699, 2078101, 2078093, 2078091, 20362627, 20362623, 20362621, 2078077, 20362619, 2078075, 20362611, 20362609, 937080144, 20362569, 20362567, 20362555, 937080117, 20362545, 20362539, 20362537, 20362523, 20362521, 20362519, 20362517, 20362513, 20362511, 20362507, 9483516, 9483484, 22656219, 20362459, 20362457, 20362445, 20362443, 20362441, 20362439, 20362437, 20362435, 20362433, 20624568, 20362421, 20362419, 10466473, 10466467, 10466455, 10466445, 5878919, 10466439, 5878913, 10466431, 5878887, 10466361, 10466333, 10466273, 10466269, 10400668, 10466181, 10400604, 10400586, 10400572, 10400564, 10466019, 10466013, 1684099, 1684095, 1684091, 10400312, 11121181, 11121179, 21279250, 21279246, 21410196, 21410190, 10400022, 18329698, 22392911, 8957970, 2666445, 2666443, 2666441, 2666439, 2666437, 2666435, 2666433, 2666431, 2666429, 2666425, 2666423, 2666407, 2666401, 2666397, 22654843, 936030071, 936030068, 2666313, 2666307, 11120443, 2666245, 15183563, 15183547, 2666169, 11120309, 4763312, 4763290, 4763276, 4763260, 4763250, 14593615, 18328962, 18328960, 20229503, 18328952, 7843191, 15838506, 5876957, 15838406, 18328746, 22391955, 19836036, 2665545, 22391821, 8956920, 18328520, 9350031, 9350017, 18328388, 18328372, 3320618, 22391565, 22391513, 18328246, 22391459, 22391399, 22653489, 15837706, 18328068, 2664951, 18328036, 8366562, 2664899, 9349563, 9349527, 22653311, 5876065, 5875871, 19835008, 5875729, 20359106, 5875641, 20359096, 20359088, 1222520, 948021078, 2664245, 19834650, 5875461, 22783732, 5875435, 22783712, 2664161, 22390447, 943040164, 943040160, 2664085, 948020873, 2664019, 15181387, 2664003, 21276192, 1615376, 1615370, 2663915, 15181271, 2663893, 15181167, 15181161, 1615134, 3711916, 3253151, 940090028, 10396001, 3252505, 16031954, 166174892, 10330271, 16949140, 18521845, 10854127, 10854065, 2662063, 10854053, 18521751, 10395247, 2661985, 2661975, 2661973, 2661913, 16817523, 16817521, 1150000442, 1150000440, 22781084, 152542334, 24288282, 22780912, 9411548, 9411546, 24288212, 24288210, 24288204, 24288202, 9411530, 24288200, 24288198, 24288196, 24288182, 24288180, 24288170, 24288166, 24288150, 24288142, 24288118, 9411430, 24288088, 16816983, 12753735, 24287980, 23304924, 24287920, 24287914, 24287874, 24287860, 24287856, 24287816, 24287814, 24287808, 20355624, 20355618, 20355616, 24287772, 20355610, 24287740, 24287730, 24287708, 12556714, 24287656, 24287654, 24287634, 24287624, 24287608, 24287586, 24287550, 24287542, 24287540, 24287532, 24287528, 24287508, 24287506, 24287486, 10721512, 24287460, 24287444, 4561084, 11311131, 10721030, 10721024, 10720992, 24155638, 10786198, 3970192, 10720322, 4559922, 946050063, 23499729, 4559698, 21533494, 23499525, 21271100, 24285668, 6263216, 6263214, 6263208, 6263204, 21532894, 6721743, 20156591, 20156563, 23957600, 6721625, 3379273, 6721595, 24416210, 6721469, 6721467, 6262694, 6262688, 948080364, 22187745, 17272524, 17272522, 17272520, 6131383, 937070239, 6131351, 937070215, 12684935, 22187647, 1216105, 10784182, 22187433, 15109499, 16944418, 2788639, 2723051, 6130881, 7900133, 2788265, 20155219, 20155209, 6261576, 20155207, 942050099, 20155185, 942050096, 20155183, 20155181, 10718000, 942050079, 17599261, 10390200, 20351629, 20351627, 20154949, 2787881, 2787857, 2787853, 2787851, 2787847, 20154869, 20154863, 2787757, 14518673, 15108491, 2787695, 2787629, 11110701, 2787619, 2787601, 18516210, 18516150, 18516148, 18516132, 18516130, 20351115, 18516074, 18516072, 18516070, 20351077, 20351027, 20351025, 936020014, 21530668, 10520588, 18515958, 20154323, 6719417, 16746415, 6719407, 6719405, 23234473, 6719393, 6719389, 20154239, 20154211, 24282978, 10520342, 10520324, 9275108, 10782418, 6129359, 6129351, 6129323, 6129321, 6129289, 6129287, 6129279, 18122362, 18122358, 23234147, 6129249, 6129245, 18515440, 18515424, 6129087, 6129083, 10978743, 18122166, 18122164, 10716566, 3179924, 6129001, 11175229, 20153633, 18384147, 20153607, 20153585, 24478941, 17663183, 944080060, 14517427, 3572721, 3572715, 12288990, 12288988, 16745403, 9274286, 9274284, 16745385, 7701392, 3572611, 3572609, 16745295, 16745283, 16745217, 20153081, 16745203, 16745201, 16745197, 10519282, 10519272, 23954139, 16745177, 23954137, 23954135, 16745173, 23954131, 16745161, 16745155, 16745141, 16745137, 3572333, 24543821, 23953995, 23953987, 1737276, 23101990, 23953951, 23953931, 18514438, 18514426, 18514368, 10518970, 10715562, 18514346, 1671591, 10518944, 11174297, 1671557, 10715516, 1212791, 1671441, 10715320, 2785405, 2785387, 22380649, 7700344, 20742002, 2785109, 2785083, 2785055, 2784987, 3112635, 1736338, 294529, 1736290, 1736286, 1342890, 23887052, 23887042, 23887028, 21331109, 23887010, 23887000, 948010100, 23886950, 23886936, 23886914, 17595457, 17595451, 17595441, 10320938, 10320930, 23886858, 23886848, 10320896, 17595347, 11566030, 940080069, 10320828, 23886712, 818025, 19037031, 23886648, 10320676, 1669925, 23886618, 23886596, 23886592, 23886590, 23886588, 10320628, 10320488, 10320486, 17594923, 17594917, 17594915, 17594913, 17594807, 11565478, 11565474, 17594771, 17594753, 17594747, 17594743, 23886156, 20216060, 11565244, 11171951, 10319966, 23361625, 3766344, 6322241, 5863417, 5863415, 5863409, 5863353, 23361347, 23361309, 6125197, 17593965, 17593963, 17593957, 17593947, 17593943, 13071882, 21263861, 6321593, 15889804, 20215046, 17593603, 17003778, 20215026, 3175602, 3175566, 20804735, 23950461, 23950455, 23950453, 21329013, 23950441, 21329001, 11301976, 23950369, 23950321, 10712044, 23950313, 23950305, 17593299, 23950287, 17593275, 23950249, 23950243, 17593247, 17593245, 17593241, 23950211, 17593217, 17593213, 6321009, 23950157, 14709580, 23950153, 23950149, 23950141, 23950137, 3502843, 3502837, 23950039, 12087975, 22770310, 22770280, 4223562, 15888868, 22770128, 10776778, 10318010, 1732736, 12087041, 3894893, 13069824, 21261787, 5860813, 22769070, 22769026, 5860669, 5860509, 5860493, 5860417, 17591314, 17591306, 17591304, 17591300, 17591296, 5860351, 17591294, 5860349, 17591292, 5860347, 17591276, 17591274, 17591272, 17591270, 5860321, 6712287, 17591266, 5860309, 17591248, 13069250, 5204874, 17591178, 17591176, 17591166, 17591164, 17591162, 17591160, 17591150, 21326697, 17591144, 17591138, 17591128, 17591076, 11561712, 3893973, 11561632, 11561626, 17590936, 2648652, 3893819, 11561432, 11561428, 11561384, 11561330, 11561288, 11561286, 23881850, 23881814, 23881808, 23881794, 17590332, 23881784, 9660432, 17590288, 17590286, 17590274, 23881726, 17590262, 17590256, 17590248, 17590242, 17590232, 17590216, 17590194, 17590166, 17590164, 17590162, 17590160, 17590156, 17590154, 17590144, 948070271, 17590140, 17590138, 17590126, 17590124, 17590120, 17590116, 17590114, 17590112, 17590110, 17590106, 17590100, 17590098, 91964, 24471330, 22636308, 23881460, 17589898, 16934528, 91736, 91728, 91710, 17589804, 17589800, 17589794, 17589792, 23881242, 20866586, 20866578, 91610, 5203362, 5203278, 11363619, 11363617, 20341941, 14181524, 14181520, 6317127, 6317033, 15950674, 6316633, 11690572, 6316609, 6316587, 6316541, 22634992, 22634976, 22634972, 6316407, 6316403, 10051924, 936010042, 17785145, 23224524, 11362501, 16933058, 24535190, 16933010, 8413323, 11362437, 11690100, 10051654, 24535004, 24534996, 7298982, 6315895, 6315887, 24534858, 25386819, 89920, 89908, 24534822, 16932622, 89868, 16932612, 89848, 89842, 8281829, 11689680, 24534720, 19619506, 8412843, 24534696, 11361941, 24534648, 19619446, 19619444, 16866931, 8412771, 8412759, 89618, 89604, 89482, 89466, 7298420, 8412509, 16932156, 8412473, 8412469, 89396, 16932132, 16932130, 8412447, 8412439, 8412433, 8412429, 8412427, 8412421, 24075517, 89340, 89332, 24075501, 89320, 1203368, 8281245, 8281237, 8281233, 8281225, 8281209, 8281179, 89174, 8281169, 8281153, 16931902, 21257261, 8281109, 8281067, 16931816, 16931814, 8281051, 16931798, 16931786, 11688836, 17586956, 11688626, 22633130, 11688600, 22633100, 11688568, 16931380, 16931378, 16931344, 16931342, 16931334, 16931332, 16931328, 16931318, 16931316, 16931312, 16931310, 24271340, 24271332, 16931298, 16931294, 16931220, 16931216, 4413830, 16931204, 16931200, 7952766, 16931194, 16931184, 16931170, 1399108, 943020354, 6314295, 6314293, 6314291, 16931120, 1399062, 6314261, 6314253, 6314239, 24270962, 24270866, 16865281, 16865279, 7690131, 8411021, 8411019, 5330713, 11228814, 5330559, 5330553, 5330521, 5330517, 5330501, 5330497, 5330493, 22763026, 5330449, 23877093, 23877091, 23877089, 16078260, 8410543, 8410537, 22762918, 8410527, 16078234, 8410525, 16078230, 8410513, 8410507, 23877001, 23876993, 24073503, 8410329, 8410323, 8410321, 8410303, 940070065, 1594533, 23876701, 11097161, 1987626, 1987622, 22762532, 22762516, 22762514, 24073215, 22762482, 22762464, 23876553, 23876515, 22631330, 5329825, 945050507, 1200924, 23876371, 23876343, 22762208, 1987288, 5329615, 5329607, 5329603, 22762160, 5329539, 20664960, 20664954, 2511419, 5329463, 5329455, 5329445, 17715742, 17715736, 17715732, 5329401, 8409587, 5329381, 20664804, 16863715, 16863713, 16863711, 4608476, 16863707, 16863701, 6181330, 16863697, 6181324, 5329353, 5329349, 16863685, 4608450, 5329301, 5329289, 4608388, 940200271, 14045479, 14045477, 14045475, 17715462, 22630494, 16863317, 17715268, 17715250, 17321957, 1200058, 16928604, 8408911, 16928590, 1986380, 1199654, 20204926, 478549, 16928080, 16928076, 16928074, 16928064, 16928048, 16928042, 16928040, 16928038, 16928036, 16928028, 16928026, 16928022, 16928006, 16928002, 16928000, 478433, 478399, 478387, 478375, 478373, 478345, 20204650, 16927778, 16927774, 16927768, 18303856, 8407617, 8931636, 1984812, 5196044, 19876042, 19876040, 22628542, 84138, 22628508, 10176633, 933120092, 21907509, 22628304, 22628250, 1984402, 4605834, 22628232, 1984344, 19875648, 1984308, 3360548, 19875614, 1984274, 19875594, 14042843, 1984154, 83568, 83562, 1984070, 1984066, 1984056, 19809845, 1984028, 2180623, 2180615, 2180613, 148978, 1983984, 1983954, 1983952, 2180535, 2180521, 1983910, 1983872, 1983864, 1983862, 1983832, 1983818, 19875140, 1983808, 1983778, 1983772, 1983770, 1983732, 3359984, 3359970, 9454793, 4605122, 3359934, 3359932, 3359930, 2180273, 3359920, 6177964, 6177960, 21906599, 3359910, 24003733, 6177938, 6177932, 24003717, 24003711, 3359858, 2180209, 2180125, 1983438, 1983366, 6177664, 6177656, 6177654, 1983328, 6177616, 3359552, 3359540, 3359536, 3359532, 3359530, 3359528, 3359526, 19874598, 3359524, 3359522, 3359512, 3359508, 3359506, 19809031, 3359470, 3359462, 3359456, 3359448, 3359436, 3359424, 1983168, 3359412, 3359410, 3359408, 1983136, 3359380, 3359376, 1983120, 3359372, 2179723, 3359354, 1983088, 3359338, 3359330, 3359328, 3162679, 3162659, 10961440, 10961438, 3162639, 3359238, 2245125, 21905923, 21905921, 21905919, 3359230, 3359228, 2245115, 7291378, 803309, 3359206, 1982928, 2244929, 2244911, 2244889, 9453829, 2244835, 10961118, 5849272, 10961078, 10961060, 10961054, 10961050, 3162239, 3162219, 1523813, 948060240, 17514570, 17514562, 19808317, 19808309, 7749578, 2179013, 2179009, 2178977, 19808151, 17514388, 3161981, 3161967, 23871299, 23871251, 23871247, 3358474, 3358472, 3358470, 10960648, 3358466, 3358464, 3358462, 3358460, 3358458, 3358456, 3358454, 3358452, 2178803, 3358450, 3358448, 3358446, 3358414, 3358406, 3358404, 3358402, 3358400, 3358398, 20266679, 2178707, 23871117, 22822466, 22822450, 6241837, 22232624, 22822422, 23936509, 2178557, 15154628, 15154626, 14826929, 867674, 8404165, 6241383, 10959910, 4340754, 20331518, 8403937, 24001489, 10959790, 10959788, 10959786, 24001449, 24001429, 10959726, 10959724, 10959722, 10959718, 10959716, 10959714, 10959712, 10959710, 10959708, 24001371, 10959706, 24001369, 21248856, 10959704, 10959702, 10959700, 10959698, 10959696, 10959694, 10959692, 10959690, 10959686, 10959682, 24001333, 24001325, 24001307, 24001305, 24001303, 24001301, 24001299, 24001297, 24001295, 5454506, 5454462, 10959484, 8403577, 22624852, 10959432, 8403519, 10959412, 801249, 801245, 801227, 801225, 27081103, 10696973, 997558, 10958950, 10958936, 10958934, 10958932, 22820850, 800737, 800735, 21247798, 944060066, 21902813, 10696141, 800175, 800167, 5846424, 14300555, 19805535, 8074548, 8074508, 22557954, 22557934, 22820056, 799877, 3880036, 10695705, 8401823, 20853659, 8205201, 8205185, 21902103, 10040048, 3879576, 3879574, 21901949, 21901927, 3879524, 19804727, 21901861, 21901853, 21901823, 21901797, 21901751, 799153, 23736735, 21901717, 21901709, 20590970, 21901681, 21901655, 22819152, 21901647, 5845310, 20853001, 21901513, 21901495, 4468915, 11284654, 5845164, 26816681, 21901469, 20852863, 1978484, 1978472, 1978466, 1978460, 1978458, 1978454, 5845074, 5845066, 10235973, 1978432, 1978420, 1978412, 5845032, 1978398, 1978396, 1978384, 1978380, 23998324, 20590452, 5844792, 5844784, 5844746, 11677399, 24194670, 21245543, 4468305, 5844512, 5844454, 5844450, 5844402, 20393385, 23997814, 23997788, 23997786, 23997780, 19803405, 23997700, 19803385, 17706229, 23997678, 17706209, 17706203, 10693713, 11283446, 16919454, 24521434, 23865990, 10234373, 23865822, 940190169, 23865704, 23865670, 23865664, 23865634, 16918636, 16918614, 16918610, 16918608, 23865422, 22227020, 16918600, 16918594, 16918584, 16918582, 16918576, 22226980, 17704903, 22226856, 16918382, 16918376, 16918342, 21309213, 16918284, 6170370, 6170368, 6170366, 6170344, 6170314, 16918212, 16918186, 21374602, 9315971, 16918126, 16918068, 5842462, 16918044, 10233219, 21374332, 21374330, 10233203, 10233195, 1385798, 9446699, 26748187, 9446605, 21374116, 20849797, 1188959, 21308467, 3154795, 3154787, 3154775, 9446215, 17703677, 17703675, 19014373, 16851644, 15999656, 18293412, 15999620, 9445935, 17703467, 1188375, 20849171, 10232221, 1319262, 1188165, 6168896, 1188145, 6168850, 6168826, 3154145, 10231979, 10231967, 18292894, 10231807, 17375232, 946020343, 10231765, 10231727, 10231713, 16916360, 10231687, 3153779, 3153777, 3153769, 1187681, 11083559, 11083555, 9445125, 9445113, 946020065, 1515225, 9903805, 946020016, 18292384, 17899145, 17899117, 17899101, 11279940, 21437959, 18292210, 17374706, 2235863, 2235861, 2235859, 17374658, 2235727, 20585770, 9510060, 11082923, 11082887, 11082845, 11082791, 11082781, 17374156, 943070154, 943070151, 943070150, 11082693, 17374122, 11082665, 16849826, 943070081, 1383296, 9509656, 17636082, 11082425, 11082407, 11082383, 11082375, 11082361, 11082345, 11082333, 20781608, 11082209, 9443785, 9443781, 9509262, 9509250, 9509166, 16849194, 20781288, 9902271, 11081893, 14751904, 24057983, 23861365, 23861359, 23861331, 23861319, 9902147, 11081791, 940120121, 940120115, 23861297, 9902121, 23861273, 23861271, 23861263, 23861255, 23861253, 23861247, 23861233, 23861223, 23861219, 9901787, 10819196, 17176098, 14685660, 14685658, 14685650, 22812061, 14685568, 10753379, 5707028, 9901313, 9901295, 9901221, 22680718, 22549631, 9311331, 9901147, 16979000, 11277334, 23991277, 23991251, 23991199, 23991189, 23991175, 7672604, 10752747, 23990963, 23990961, 23990957, 23990895, 23990831, 18354717, 23990785, 23990783, 23990719, 23990707, 23990679, 23990673, 23990645, 23990641, 23990611, 23990603, 16978242, 4264254, 23990589, 23990587, 23990577, 23990575, 23990569, 23990567, 23990563, 23990555, 23990551, 23990543, 23990533, 23990531, 23990529, 23990503, 23990501, 23990497, 23990491, 23990481, 23990457, 23990455, 23990451, 23990449, 23990445, 23990439, 21762206, 23990421, 23990419, 23990417, 23990399, 23990395, 23990391, 23990385, 23990353, 23990341, 23990317, 23990309, 23990307, 13635603, 23990275, 23990273, 23990267, 23990265, 23990227, 23990221, 23990215, 23990213, 23990211, 23990209, 23990207, 23990205, 23990203, 23990201, 23990193, 23990183, 23990181, 23990177, 23990171, 23990163, 23990161, 23990155, 23990153, 10817412, 23990143, 23990137, 23990133, 23990129, 10817390, 23990087, 23990081, 23990067, 23990065, 23990059, 23990055, 23990045, 23990039, 23990001, 23989999, 23989983, 23989971, 265903, 23989839, 23989837, 23989835, 23989833, 23989825, 23989817, 23989811, 23989809, 17763833, 10817014, 23399823, 19074421, 17370434, 12061706, 13503438, 12585923, 22678418, 22678396, 16976742, 11078421, 11078411, 22678204, 11078327, 11078321, 11078313, 27724453, 27724417, 27724405, 11078241, 11078223, 11340339, 11340289, 7866822, 11340215, 11340211, 7866736, 10815856, 20318300, 7866402, 7866388, 11077611, 1574638, 1574632, 8193663, 11077153, 7865874, 10814980, 20121070, 10814942, 1574354, 1574352, 1574338, 1574334, 18351525, 7865746, 10814860, 1574264, 18351479, 9897320, 7865652, 20120830, 18351285, 18351283, 17171610, 7865470, 17171570, 22676586, 14156900, 14746658, 22676514, 8913907, 14746572, 8192945, 8192941, 8192897, 17695607, 8192879, 8192827, 937100074, 11338517, 14746358, 19858110, 8192677, 19858066, 11338317, 23134781, 940180015, 8192539, 14352922, 8192519, 23921150, 4915706, 4915704, 8192499, 8192495, 22675798, 4915542, 9437501, 4915502, 23920844, 9437371, 10682508, 8192107, 2556005, 7733311, 7733307,]

**Subsetting NWM output using nwmurl**

This code utilizes nwmurl library to create a subset of NWM output based on output type, date range, data source.
Instructions on subsetting NWM data using nwmrul can be found here: https://github.com/CIROH-UA/nwmurl

In this example, we are generating a subset of NWM output for operational medium range zarr files for 01/11/2022.
This code snippet initializes an empty list called `files` to store URLs. 
It then specifies the name of a file, "filenamelist.txt," which contains a list of URLs. 
The code opens this file in read mode, reads the URLs line by line, removes leading and trailing whitespace from each line, and appends each URL to the `files` list. 

In [ ]:
# Initialize an empty list to store the URLs
files = []

# Define the name of the file containing the URLs
filename = "filenamelist.txt"
start_date = "202201110000"
end_date   = "202201110000"
#end_date   = "202201200000"
fcst_cycle = [0]
lead_time = None
varinput = 1
geoinput = 1
runinput = 2
urlbaseinput = 9
meminput = 1
write_to_file = True

files = nwmurl.generate_urls_operational(
    start_date, end_date, fcst_cycle,
    lead_time,
    varinput,
    geoinput,
    runinput,
    urlbaseinput,
    meminput,
    write_to_file
)

***Parallel run***

This code snippet utilizes the `joblib` library to create a parallel processing pool using threading for dynamic settings. It processes a list of `files` using the `process_file` function in parallel, passing each `file_url` and the `feature_ids` list as arguments. The results are stored in `result_values` and sorted based on the first element of each result tuple. The extracted streamflow values are then stored in the `Streamflow` list, and the associated timestamps are stored in the `Timestamp` list, which can be further used for analysis or visualization.

In [ ]:
%%time
# Create a Parallel processing pool using joblib with dynamic settings
with joblib.parallel_backend("threading", n_jobs=16):  # Use threads for parallel processing
    results = joblib.Parallel()(joblib.delayed(process_file)(file_url, feature_ids) for file_url in files)

***Serial run***

In [ ]:
%%time

# Run in serial (no parallel processing)
results = [process_file(file_url, feature_ids) for file_url in files]


In [ ]:
# Filter out unsuccessful downloads
results = [result for result in results if all(val is not None for val in result)]    
results = sorted(results, key=lambda x: x[1][0])
Streamflow = [item[0] for item in results]
Timestamp = [item[1] for item in results]

***Visualization***

This code snippet creates separate plots for each feature ID in the `feature_ids` list. For each feature ID, it plots the time series of the extracted streamflow values using Matplotlib. The streamflow values are stored in the `Streamflow` list, and each plot is labeled with the corresponding feature ID. The x-axis represents time in hours, and the y-axis represents streamflow values. Finally, the code displays all the separate plots.


In [ ]:
# Create separate plots for each feature ID
    # Plot the time series of the extracted streamflow values for the current feature ID using Matplotlib
plt.figure()
plt.plot(Streamflow, label=[f"Feature ID {feature_id}" for feature_id in feature_ids])
plt.xlabel(f"Time(hours after {min(Timestamp)})")
#plt.xlabel("Time(hours)")
plt.ylabel("Streamflow")
plt.title(f"Streamflow Time Series")
plt.legend()
# Show all the separate plots
plt.show()

# 2: Downloading NC files
This code snippet initializes an empty list called `files` to store URLs. It then specifies the name of a file, "filenamelist.txt," which contains a list of URLs. The code opens this file in read mode, reads the URLs line by line, removes leading and trailing whitespace from each line, and appends each URL to the `files` list. Finally, it initializes an empty dictionary called `extracted_values_dict`,  intended to store extracted values associated with specific feature IDs.

In [ ]:
#Medium_Range
import nwmurl

start_date = "202201110000"
end_date   = "202201110000"
#end_date   = "202201200000"
fcst_cycle = [0]
lead_time = None
varinput = 1
geoinput = 1
runinput = 2
urlbaseinput = 3
meminput = 1
write_to_file = True

file_urls = nwmurl.generate_urls_operational(
    start_date, end_date, fcst_cycle,
    lead_time,
    varinput,
    geoinput,
    runinput,
    urlbaseinput,
    meminput,
    write_to_file
)

## Parallel Processing of NetCDF Files

The provided code snippet demonstrates the parallel processing of NetCDF files containing streamflow data using the `joblib` library. This script is designed to efficiently download, process, and extract streamflow and time information from multiple NetCDF files.

## Features:

1. **Feature Selection:** The user can specify a list of feature IDs to extract from the NetCDF files. In the provided example, feature IDs `[8153461, 8153027, 18210860]` are used.

2. **Error Handling:** The code includes error handling mechanisms, logging errors when issues occur during streamflow or time data selection, file download, or NetCDF file processing.

3. **Parallel Processing:** The `joblib` library is employed for parallel processing, utilizing multiple cores for faster execution. The number of parallel jobs is set to the maximum available using `n_jobs=-1`.

4. **Functions for Data Selection:** Functions `select_flow` and `select_time` are defined for extracting streamflow and time data, respectively, from the opened NetCDF dataset.

5. **File Processing:** The script downloads NetCDF files from specified URLs, saves them locally, opens them with xarray, extracts relevant data, and then closes and removes the files.

6. **Results Filtering:** Unsuccessful downloads are filtered out, and the final results are separated into lists of streamflow (`sflowold`) and time (`timeold`) data.

7. **Timing Information:** The `%%time` magic command is used to measure the execution time of the entire code block.

This script is a practical example of handling NetCDF files in a parallelized manner, suitable for scenarios where processing multiple files concurrently is essential for performance optimization.


In [ ]:
%%time
import xarray as xr
import requests
from joblib import Parallel, delayed
import os

# Define the list of feature IDs you want to extract
feature_ids = [8153461]

# Define a function to select streamflow data from a dataset
def select_flow(ds, feature_id):
    try:
        cords = ds.streamflow.sel(feature_id=feature_id)
        return cords.values
    except Exception as e:
        print(f"Error selecting streamflow data: {e}")
        return None

# Define a function to select time
def select_time(ds):
    try:
        dstime = ds.time
        return dstime.values
    except Exception as e:
        print(f"Error selecting time data: {e}")
        return None

def process_file(file_url, feature_id):
    try:
        # Download the file from the URL
        response = requests.get(file_url)

        if response.status_code == 200:
            # Save the file to a local location with the same name
            filename = file_url.split('/')[-3] + file_url.split('/')[-1]
            with open(filename, 'wb') as nc_file:
                nc_file.write(response.content)

            try:
                # Open the downloaded NetCDF file using xarray
                with xr.open_dataset(filename) as ds:
                    streamflow_value = select_flow(ds, feature_id)
                    time_value = select_time(ds)
            except Exception as e:
                print(f"Error processing NetCDF file: {e}")
                return None, None
            finally:
                # Close the dataset to release the file lock
                ds.close()

            # Remove the file after closing the dataset
            os.remove(filename)
            return streamflow_value, time_value
        else:
            print(f"Failed to download file from {file_url}")
            return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None


with Parallel(n_jobs=16) as parallel:
    results = parallel(delayed(process_file)(file_url, feature_ids) for file_url in file_urls)

# Filter out unsuccessful downloads
results = [result for result in results if all(val is not None for val in result)]

# Separate the results into streamflow and time lists
sflowold, timeold = zip(*results)

***Serial run***

In [ ]:
%%time
# Run in serial (no parallel processing)
results = [process_file(file_url, feature_ids) for file_url in file_urls]


***Visualization***

In [ ]:
# Create separate plots for each feature ID
    # Plot the time series of the extracted sflowold values for the current feature ID using Matplotlib
plt.figure()
plt.plot(sflowold, label=[f"Feature ID {feature_id}" for feature_id in feature_ids])
plt.xlabel(f"Time(hours after {min(timeold)})")
#plt.xlabel("Time(hours)")
plt.ylabel("sflowold")
plt.title(f"sflowold Time Series")
plt.legend()
# Show all the separate plots
plt.show()